# Framework for QCD global analysis
<img style="float: left;" src="logos/jam.jpg"/>


### Author: Nobuo Sato (Jefferson Lab - Theory)

In [1]:
import sys,os,time
import numpy as np
import pandas as pd
import copy
import threading

#--matplotlib
import matplotlib
#matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
#matplotlib.rc('text',usetex=True)
import pylab  as py
from matplotlib.lines import Line2D

#--scipy 
from scipy.integrate import fixed_quad
from scipy.integrate import quad
from scipy.special import gamma

#--mpmath
from mpmath import fp

#--from scipy
from scipy.optimize  import minimize,leastsq
from scipy.optimize  import least_squares

#--jupyter widgets
from IPython.display import display,clear_output
import ipywidgets as widgets


### Attention:
These are all the depedencies. If you don't have them installed in your python env, use `pip install ...`. e.g. `pip install mpmath`

In [2]:
class AUX:

    def __init__(self):

        #--set_constants 
    
        self.CA=3.0
        self.CF=4.0/3.0
        self.TR=0.5
        self.TF=0.5
        self.euler=fp.euler 

        #--set_masses

        self.me   = 0.000511
        self.mm   = 0.105658
        self.mt   = 1.77684
        self.mu   = 0.055
        self.md   = 0.055
        self.ms   = 0.2
        self.mc   = 1.28
        self.mb   = 4.18

        self.mZ   = 91.1876
        self.mW   = 80.398
        self.M    = 0.93891897
        self.Mpi  = 0.13803
        self.Mk   = 0.493677
        self.Mdelta = 1.232

        self.me2   = self.me**2 
        self.mm2   = self.mm**2 
        self.mt2   = self.mt**2
        self.mu2   = self.mu**2  
        self.md2   = self.md**2  
        self.ms2   = self.ms**2  
        self.mc2   = self.mc**2  
        self.mb2   = self.mb**2  
        self.mZ2   = self.mZ**2  
        self.mW2   = self.mW**2  
        self.M2    = self.M**2  
        self.Mpi2  = self.Mpi**2  
        self.Mdelta2=self.Mdelta**2

        #--set_couplings       
        
        self.c2w   = self.mW2/self.mZ2
        self.s2w   = 1.0-self.c2w
        self.s2wMZ = 0.23116
        self.c2wMZ = 1.0 - self.s2wMZ
        self.alfa  = 1/137.036
        self.alphaSMZ = 0.118
        self.GF = 1.1663787e-5   # 1/GeV^2



### Attention:
- we will use `conf` as a dictionary to store class instances
- classes defined below will use mostly pre-loaded class instances at `conf`

In [3]:
conf={}
conf['aux']=AUX()
print(conf['aux'].M)

0.93891897


## $\alpha_{\rm EM}$ 

In [4]:
class EWEAK:

    def __init__(self):
        self.aux=conf['aux']
        self.ml2 = np.array([self.aux.me2,self.aux.mm2,self.aux.mt2])
        self.mq2 = np.array([self.aux.mu2,self.aux.md2,self.aux.ms2,self.aux.mc2,self.aux.mb2])
        self.eq  = np.array([2./3, -1./3, -1./3, 2./3, -1./3])
        self.gvl = (-1.0)*(-1.0 + 4*self.aux.s2w)/2
        self.i3q = np.array([0.5, -0.5, -0.5, 0.5, -0.5])
        self.gvq = self.i3q - 2*self.eq*self.aux.s2w
        self.alfa = self.aux.alfa
        self.logc2w = np.log(self.aux.c2w)
        self.alpha = {}
        self.kappa = {}

    def get_PIlepton(self,Q2,flag):
        #--lepton loop contribution
        if Q2 >= 2.e-8:
            u = 4*self.ml2/Q2
            su = np.sqrt(1.0+u)
            tmp = ( (1.0-u/2)*su*np.log((su+1.0)/(su-1.0)) + u - 5.0/3 )/3
        else:
            tmp = Q2/15./self.ml2
        if   flag=='gg': return np.sum(tmp)
        elif flag=='gZ': return np.sum((-np.log(self.aux.mZ2/self.ml2)/3 + tmp)*self.gvl)

    def get_PIquark(self,Q2,flag):
        #--quark loop contribution
        if Q2 >= 2.e-5:
            u = 4*self.mq2/Q2
            su = np.sqrt(1.0+u)
            tmp = ( (1.0-u/2)*su*np.log((su+1.0)/(su-1.0)) + u - 5.0/3 )/3
        else:
            tmp = Q2/15./self.mq2
        if   flag=='gg': return np.sum(3*tmp*self.eq**2)
        elif flag=='gZ': return np.sum(3*(-np.log(self.aux.mZ2/self.mq2)/3 + tmp)*self.gvq*self.eq)

    def get_kapb(self,Q2):
        #--boson loop contribution to kappa
        if Q2 >= 1.0:
            z = self.aux.mW2/Q2
            p = np.sqrt(1.0 + 4*z)
            logp = np.log((p+1.0)/(p-1.0))
            PIb = 1./18 \
              + ( -1.0 - 42*self.aux.c2wMZ ) * self.logc2w/12 \
              - ( self.aux.c2wMZ*(7.0 - 4*z) + (1.0 + 4*z)/6 ) * ( p*logp/2 - 1.0 ) \
              - z * ( 0.75 - z + p*(z-1.5)*logp + (2.0-z)*z*logp**2 )
            return -(self.alfa/np.pi)/(2*self.aux.s2wMZ) * PIb
        else:      
            return - self.alfa*(-43.0 - 444*self.aux.c2wMZ)*Q2 / (1440*self.aux.mW2*np.pi*self.aux.s2wMZ) \
             + self.alfa*(-16.0 - 12*self.aux.c2wMZ + 3*self.logc2w + 126*self.aux.c2wMZ*self.logc2w) \
               / (72*np.pi*self.aux.s2wMZ)

    def get_alpha(self,Q2):
        #--running of alpha_em
        if Q2 not in self.alpha:
            dalpl = (self.alfa/np.pi) * self.get_PIlepton(Q2,'gg')
            dalpq = (self.alfa/np.pi) * self.get_PIquark(Q2,'gg')
            self.alpha[Q2] = self.alfa/(1.0-dalpl-dalpq)
        return self.alpha[Q2]

    def get_sin2w(self,Q2):
        if Q2 not in self.kappa:
            kapl = -(self.alfa/np.pi)/(2*self.aux.s2wMZ) * self.get_PIlepton(Q2,'gZ')
            kapq = -(self.alfa/np.pi)/(2*self.aux.s2wMZ) * self.get_PIquark(Q2,'gZ')
            kappa = 1.0 + kapl + kapq + self.get_kapb(Q2)
            self.kappa[Q2] = kappa
        return self.kappa[Q2]*self.aux.s2wMZ


### Example

In [5]:
conf={}
conf['aux']=AUX()
eweak=EWEAK()
mu2=10.0
print(eweak.get_alpha(mu2))

0.0074977699507687255


### Comment:
- We initialize `AUX` inside `conf` since `EWEAK` calls internally aux from conf
- In this example we initilize `eweak` not as entry of `conf` bacause is not needed. Later, there will be some classes that neeeds `eweak` loaded as entry of `conf`  


### Exercise 1 (time: 5 mins)

- plot $\alpha_{\rm EM}$ as a function of $\mu^2 \in (m_e^2, 10^4)$
- include vertical lines indicating the mass thresholds $m_{e}^2,~m_{\mu}^2,~m_{\tau}^2$

## $\alpha_S$ running

In [6]:
class ALPHAS:

    def __init__(self,order=1):
        """
        order=0: 1-loop
        order=1: 2-loops
        """
        if 'order' in conf: order=conf['order']
        self.order=order
        self.mu20=1
        self.setup()
  
    def setup(self):
        aux = conf['aux']
  
        #--setup the beta function
        self.beta=np.zeros((7,3))
        for Nf in range(3,7): 
            self.beta[Nf,0]=11.0-2.0/3.0*Nf 
            self.beta[Nf,1]=102.-38.0/3.0*Nf 
            self.beta[Nf,2]=2857.0/2.0-5033.0/18.0*Nf+325.0/54.0*Nf**2 
  
        # uses alphaS(mZ)--> backwards evolution
        self.aZ  = conf['aux'].alphaSMZ/(4*np.pi)
        self.ab=self.evolve_a(self.aZ,aux.mZ2,aux.mb2,5)
        self.ac=self.evolve_a(self.ab,aux.mb2,aux.mc2,4)
        self.a0=self.evolve_a(self.ac,aux.mc2,self.mu20,3)
  
        # we will store all Q2 values of alphaS 
        self.storage={}
  
        
    def get_Nf(self,mu2):
        aux = conf['aux']
        Nf=3
        if mu2>=(aux.mc2): Nf+=1
        if mu2>=(aux.mb2): Nf+=1
        return Nf
  
    def beta_func(self,a,Nf):
        betaf = -self.beta[Nf,0]
        if self.order>=1: betaf+=-a*self.beta[Nf,1]
        if self.order>=2: betaf+=-a*self.beta[Nf,2]
        return betaf*a**2
  
    def evolve_a(self,a0,mu20,mu2,Nf):
        # Runge-Kutta implemented in pegasus  
        LR = np.log(mu2/mu20)/20.0
        a=a0
        for k in range(20):
            XK0 = LR * self.beta_func(a,Nf)
            XK1 = LR * self.beta_func(a + 0.5 * XK0,Nf)
            XK2 = LR * self.beta_func(a + 0.5 * XK1,Nf)
            XK3 = LR * self.beta_func(a + XK2,Nf)
            a+= (XK0 + 2.* XK1 + 2.* XK2 + XK3) * 0.166666666666666
        return a
  
    def get_a(self,mu2):
        aux = conf['aux']
  
        if mu2 not in self.storage:

            if aux.mb2<=mu2:
                mu20,a0,Nf=aux.mb2,self.ab,5
            elif aux.mc2<=mu2 and mu2<aux.mb2: 
                mu20,a0,Nf=aux.mc2,self.ac,4
            elif mu2<aux.mc2:
                mu20,a0,Nf=self.mu20,self.a0,3
            self.storage[mu2]=self.evolve_a(a0,mu20,mu2,Nf)
  
        return self.storage[mu2]
  
    def get_alphaS(self,mu2):
        return self.get_a(mu2)*4*np.pi
  

### Example

In [7]:
conf={}
conf['aux']=AUX()
aS=ALPHAS(order=1)
mu2=conf['aux'].mZ2
print(aS.get_alphaS(mu2))

0.11799999990923955


### Comments
- The QCD running coupling is given by the following RGE

  $\frac{da_S (\mu^2)}{d\ln \mu^2} =\beta(\mu^2)= -\sum_{k=0}^{\infty} \beta_k a_s^{k+2}(\mu^2)$

  with 
  
  $a_s(\mu^2) = \frac{\alpha_S(\mu^2)}{4\pi}$
  
- To solve the RGE we need to have a boundary condition. We shall use the known value of $\alpha_S(m_Z)=0.118$
- **Mass thresholds** are defined at the points $\mu=m_c,m_b$
- The beta function is discontinuous at at the mass thresholds due to $N_f=3,4,5$
- To solve the RGE in the range $1\leq\mu^2\leq\infty$ we need to build boundary conditions for each region with the same $N_f$  (see the `if` statements of the function ``get_a(mu2)``)
- The boundary conditions are computed at the mass thresholds

In [8]:
print('alphaS(mZ)/4pi=',aS.aZ)
print('alphaS(mb)/4pi=',aS.ab)
print('alphaS(mc)/4pi=',aS.ac)
print('alphaS(1)/4pi=',aS.a0)

alphaS(mZ)/4pi= 0.009390141642421825
alphaS(mb)/4pi= 0.01779207433483575
alphaS(mc)/4pi= 0.02964941971713454
alphaS(1)/4pi= 0.03538272992118547


### Exercise 2.A (time: 5 mins)

- plot $\alpha_{\rm S}$ as a function of $\mu^2 \in (1, 10^4)$  
- make the plot using 1-loop and 2-loops  
- include vertical lines indicating the mass thresholds $m_{c}^2,~m_{b}^2$ and $m_{Z}^2$

### Exercise 2.B (time: 5 mins)

- plot $\beta(\mu^2)$ as a function of $\mu^2 \in (1, 10^4)$  
- using 2-loops (order=1)  
- include vertical lines indicating the mass thresholds $m_{c}^2,~m_{b}^2$ and $m_{Z}^2$
- **Hint**:
  + for a given $\mu^2$, compute `a` via `get_a`  and `Nf` via `get_Nf`
  + use the function `beta_func(a,Nf)` to get the numerical value of the beta function

## Mellin transforms

In [9]:
class MELLIN:

    def __init__(self,npts=8,c=None):
  
        #--gen z and w values along coutour
        x,w=np.polynomial.legendre.leggauss(npts)
        znodes=[0,0.1,0.3,0.6,1.0,1.6,2.4,3.5,5,7,10,14,19,25,32,40,50,63]
  
        Z,W,JAC=[],[],[]
        for i in range(len(znodes)-1):
            a,b=znodes[i],znodes[i+1]
            Z.extend(0.5*(b-a)*x+0.5*(a+b))
            W.extend(w)
            JAC.extend([0.5*(b-a) for j in range(x.size)])
        Z=np.array(Z)
        #--globalize
        self.W=np.array(W)
        self.Z=Z
        self.JAC=np.array(JAC)
        #--gen mellin contour
        if c==None: c=1.9 
        phi=3.0/4.0*np.pi
        self.N=c+Z*np.exp(complex(0,phi)) 
        self.phase= np.exp(complex(0,phi))
          
    def invert(self,x,F):
        return np.sum(np.imag(self.phase * x**(-self.N) * F)/np.pi * self.W * self.JAC)


### Example

In [10]:
mell=MELLIN(8,c=1.9)

f=lambda x: x
F=lambda N: 1/N

x=0.1
print(f(x))
print(mell.invert(x,F(mell.N)))

0.1
0.9999999999999942


### comments:
- We implement inverse mellin trasform using **gaussian quadrature** with fixed points along the complex contour
- Because of this, we can implement the mellin transform not as a function but as an array of complex values which gives the same results 

In [11]:
mell=MELLIN(8,c=1.9)

f=lambda x: x
F=1/mell.N

x=0.1
print(f(x))
print(mell.invert(x,F))

0.1
0.9999999999999942


- Notice $F$ has a pole at $N=0$. We use $c=1.9$ which is to the right of the pole. 
- If we chose a $c<0$ it won't work 

In [12]:
mell=MELLIN(8,c=-1)

f=lambda x: x
F=1/mell.N

x=0.1
print(f(x))
print(mell.invert(x,F))

0.1
-8.902104517405883e-18


### Exercise 3.A (time: 5 mins)

- Lets try $f(x)=x^a(1-x)^b$ with $a=-0.5$ and $b=3$. 
- Plot $xf(x)$ vs. $x$ and the inverse mellin transform for $0<x<1$   (use log scale for the x axis)
- **Hint**: the mellin transform of $f$ is  $F(N)=\Gamma(N+a)\Gamma(b+1)/\Gamma(N+a+b+1)$
- **Hint**: Gamma function is available via `gamma(...)`
- **Attention**: the pole of $F$ is at $N=-a$. Choose $c>-a$

### Exercise 3.B (time: 10 mins)

Consider the convolution

$\sigma(z)=\int_z^1 \frac{dx}{x}f(x)g\left(\frac{z}{x}\right)$

The mellin transform is  

$\Sigma(N) = F(N)G(N)$

with 

$F(N)=\int_0^1 x^{N-1} f(x)$

$G(N)=\int_0^1 x^{N-1} g(x)$

Using 

$f(x)=x^a(1-x)^b$ with $a=-0.5$ and $b=3$. 

$g(x)=x^c(1-x)^d$ with $c=1.0$ and $d=3$

- Plot: $z\sigma(z)$ vs. $z$
- Plot: $z{\cal M}^{-1}(\Sigma)$ vs. $z$
- use $0<z<1$

## Splitting kernels

In [13]:
import warnings
warnings.filterwarnings("ignore", message="divide by zero encountered in divide")
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", message="invalid value encountered in multiply")
warnings.filterwarnings("ignore", message="invalid value encountered in subtract")
warnings.filterwarnings("ignore", message="invalid value encountered in add")
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide")
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide")

In [14]:
class KERNELS:

    def __init__(self):

        self.D={}
        self.D['N']      = conf['mellin'].N
        self.D['nflav']  = 6+1
        self.D['norder'] = 2+1
        self.D['Nsize']  = conf['mellin'].N.size
        self.set_abbreviations()
        self.LO_unpolarized_splitting_functions()
        self.NLO_unpolarized_splitting_functions()
        self.load_unpolarized_spl()

    def set_abbreviations(self):

        D=self.D

        D['CA']=3
        D['CF']=4/3
        D['TR']=0.5

        zeta2=fp.zeta(2)
        zeta3=fp.zeta(3)

        N   = D['N']
        N1  = N + 1
        N2  = N + 2
        N3  = N + 3
        N4  = N + 4
        N5  = N + 5
        N6  = N + 6
        N7  = N + 7
        N8  = N + 8

        NM  = N - 1
        NM2 = N - 2
        NMS = NM**2
        NMT = NM**3

        NS  = N**2
        NT  = N**3
        NFO = N**4
        NFI = N**5
        NSI = N**6
        NSE = N**7
        NE  = N**8
        NN  = N**9

        N1S = N1**2
        N1T = N1**3

        N2S = N2**2
        N2T = N2**3

        psi=lambda i,_N: fp.psi(i,complex(_N.real,_N.imag))

        S1f = lambda _N: fp.euler + psi(0,_N+1)
        S2f = lambda _N: zeta2 - psi(1,_N+1)

        S1 = np.array([S1f(n) for n in N])
        D['S1']=S1
        S2 = np.array([S2f(n) for n in N])

        S11 = S1  + 1/N1 
        S12 = S11 + 1/N2
        S13 = S12 + 1/N3
        S14 = S13 + 1/N4
        S15 = S14 + 1/N5
        S16 = S15 + 1/N6
        S21 = S2  + 1/N1S

        SPMOM = 1.0000 * (zeta2 - S1 /  N) / N\
               -0.9992 * (zeta2 - S11/ N1) / N1\
               +0.9851 * (zeta2 - S12/ N2) / N2\
               -0.9005 * (zeta2 - S13/ N3) / N3\
               +0.6621 * (zeta2 - S14/ N4) / N4\
               -0.3174 * (zeta2 - S15/ N5) / N5\
               +0.0699 * (zeta2 - S16/ N6) / N6 
              

        D['PSI(N/2)']    =np.array([psi(0,n) for n in N/2])
        D['PSI(N1/2)']   =np.array([psi(0,n) for n in N1/2])
        D['PSI(N2/2)']   =np.array([psi(0,n) for n in N2/2])
        D['DPSI(N1/2,1)']=np.array([psi(1,n) for n in N1/2])
        D['DPSI(N2/2,1)']=np.array([psi(1,n) for n in N2/2])
        D['DPSI(N1/2,2)']=np.array([psi(2,n) for n in N1/2])
        D['DPSI(N2/2,2)']=np.array([psi(2,n) for n in N2/2])
      
        G1  = np.array([0.5*(psi(1,(n+1)/2)-psi(1,n/2))     for n in N])
        G11 = np.array([0.5*(psi(1,(n+2)/2)-psi(1,(n+1)/2)) for n in N])

        SLC    = - 5/8 * zeta3
        SLV    = - zeta2/2*(D['PSI(N1/2)']-D['PSI(N/2)'])+S1/NS+SPMOM
        SSCHLM = SLC - SLV
        SSTR2M = zeta2 - D['DPSI(N1/2,1)']
        SSTR3M = 0.5 * D['DPSI(N1/2,2)'] + zeta3
        SSCHLP = SLC + SLV
        SSTR2P = zeta2 - D['DPSI(N2/2,1)']
        SSTR3P = 0.5 * D['DPSI(N2/2,2)'] + zeta3

        
        D['PNMA']=(16*S1*(2*N+1)/(NS*N1S)+16*(2*S1-1/(N*N1))*(S2-SSTR2M)+64*SSCHLM+24*S2-3-8*SSTR3M-8*(3*NT+NS-1)/(NT*N1T)+16*(2*NS+2*N+1)/(NT*N1T))*(-1/2)
        D['PNPA']=(16*S1*(2*N+1)/(NS*N1S)+16*(2*S1-1/(N*N1))*(S2-SSTR2P)+64*SSCHLP+24*S2-3-8*SSTR3P-8*(3*NT+NS-1)/(NT*N1T)-16*(2*NS+2*N+1)/(NT*N1T))*(-1/2)
        D['PNSB']=(S1*(536/9+8*(2*N+1)/(NS*N1S))-(16*S1+52/3-8/(N*N1))*S2-43/6-(151*NFO+263*NT+97*NS+3*N+9)*4/(9*NT*N1T))*(-1/2)
        D['PNSC']=(-160/9*S1+32/3*S2+4/3+16*(11*NS+5*N-3)/(9*NS*N1S))*(-1/2)
        D['PPSA']=(5*NFI+32*NFO+49*NT+38*NS+28*N+8)/(NM*NT*N1T*N2S)*2

        D['PQGA']=(-2*S1*S1+2*S2-2*SSTR2P)*(NS+N+2)/(N*N1*N2)+(8*S1*(2*N+3))/(N1S*N2S)+2*(NN+6*NE+15*NSE+25*NSI+36*NFI+85*NFO+128*NT+104*NS+64*N+16)/(NM*NT*N1T*N2T)
        D['PQGB']=(2*S1*S1-2*S2+5)*(NS+N+2)/(N*N1*N2)-4*S1/NS+(11*NFO+26*NT+15*NS+8*N+4)/(NT*N1T*N2)
        D['PGQA']=(-S1*S1+5*S1-S2)*(NS+N+2)/(NM*N*N1)-2*S1/N1S-(12*NSI+30*NFI+43*NFO+28*NT-NS-12*N-4)/(2*NM*NT*N1T)
        D['PGQB']=(S1*S1+S2-SSTR2P)*(NS+N+2)/(NM*N*N1)-S1*(17*NFO+41*NS-22*N-12)/(3*NMS*NS*N1)+(109*NN+621*NE+1400*NSE+1678*NSI+695*NFI-1031*NFO-1304*NT-152*NS+432*N+144)/(9*NMS*NT*N1T*N2S)
        D['PGQC']=(S1-8/3)*(NS+N+2)/(NM*N*N1)+1/N1S
        D['PGQC']=4/3*D['PGQC']

        D['PGGA']=-(2*NFI+5*NFO+8*NT+7*NS-2*N-2)*8*S1/(NMS*NS*N1S*N2S)-67/9*S1+8/3-4*SSTR2P*(NS+N+1)/(NM*N*N1*N2)+2*S1*SSTR2P-4*SSCHLP+1/2*SSTR3P+(457*NN+2742*NE+6040*NSE+6098*NSI+1567*NFI-2344*NFO-1632*NT+560*NS+1488*N+576)/(18*NMS*NT*N1T*N2T)
        D['PGGB']=(38*NFO+76*NT+94*NS+56*N+12)*(-2)/(9*NM*NS*N1S*N2)+20/9*S1-4/3
        D['PGGC']=(2*NSI+4*NFI+NFO-10*NT-5*NS-4*N-4)*(-2)/(NM*NT*N1T*N2)-1
        
        

    def LO_unpolarized_splitting_functions(self):

        D=self.D
        D['P0QQ']=np.zeros((D['nflav'],D['Nsize']),dtype=complex)
        D['P0QG']=np.zeros((D['nflav'],D['Nsize']),dtype=complex)
        D['P0GQ']=np.zeros((D['nflav'],D['Nsize']),dtype=complex)
        D['P0GG']=np.zeros((D['nflav'],D['Nsize']),dtype=complex)  
        D['P0'] = np.zeros((D['nflav'],2,2,D['Nsize']),dtype=complex)

        N=D['N']
        for Nf in range(3,D['nflav']):

            D['P0QQ'][Nf]=4/3*(3+2/N/(N+1)-4*D['S1'])
            D['P0QG'][Nf]=2*(N**2+N+2)/(N*(N+1)*(N+2))*Nf
            D['P0GQ'][Nf]=8/3*(N**2+N+2)/(N-1)/N/(N+1)
            D['P0GG'][Nf]=3*(11/3+4/N/(N-1)+4/(N+1)/(N+2)-4*D['S1'])-2/3*Nf

            D['P0'][Nf,0,0] = D['P0QQ'][Nf] 
            D['P0'][Nf,0,1] = D['P0QG'][Nf]
            D['P0'][Nf,1,0] = D['P0GQ'][Nf]
            D['P0'][Nf,1,1] = D['P0GG'][Nf]
 
    def NLO_unpolarized_splitting_functions(self):
        D=self.D

        CF=D['CF']
        CA=D['CA']
        TR=D['TR']

        D['P1NSP']=np.zeros((D['nflav'],D['Nsize']),dtype=complex)
        D['P1NSM']=np.zeros((D['nflav'],D['Nsize']),dtype=complex)
        D['P1']=np.zeros((D['nflav'],2,2,D['Nsize']),dtype=complex)

        for NF in range(3,D['nflav']):

            D['P1NSP'][NF]=  CF*((CF-CA/2)*D['PNPA'] + CA* D['PNSB'] + TR*NF*D['PNSC'])
            D['P1NSM'][NF]=  CF*((CF-CA/2)*D['PNMA'] + CA* D['PNSB'] + TR*NF*D['PNSC'])

            D['P1'][NF,0,0]=D['P1NSP'][NF]+TR*NF*CF*D['PPSA']*4
            D['P1'][NF,0,1]=TR*NF*(CA*D['PQGA']+CF*D['PQGB'])*4
            D['P1'][NF,1,0]=(CF*CF*D['PGQA']+CF*CA*D['PGQB']+TR*NF*CF*D['PGQC'])*4
            D['P1'][NF,1,1]=(CA*CA*D['PGGA']+TR*NF*(CA*D['PGGB']+CF*D['PGGC']))*4

    def load_unpolarized_spl(self):
        D=self.D
        Nsize=D['N'].size
        nflav=D['nflav']
        norder=D['norder']
 
        #--initialize flav composed splitting functions arrays
        self.PNSP=np.zeros((nflav,norder,Nsize),dtype=complex)
        self.PNSM=np.zeros((nflav,norder,Nsize),dtype=complex)
        self.PNSV=np.zeros((nflav,norder,Nsize),dtype=complex)
        self.P   =np.zeros((nflav,norder,2,2,Nsize),dtype=complex)

        for Nf in range(3,nflav):

            #--LO unpolarized
            self.PNSP[Nf,0] = D['P0QQ'][Nf] 
            self.PNSM[Nf,0] = D['P0QQ'][Nf] 
            self.PNSV[Nf,0] = D['P0QQ'][Nf] 
            self.P[Nf,0]    = D['P0'][Nf]

            #--NLO unpolarized
            self.PNSP[Nf,1] = D['P1NSP'][Nf] 
            self.PNSM[Nf,1] = D['P1NSM'][Nf]
            self.PNSV[Nf,1] = D['P1NSM'][Nf]
            self.P[Nf,1]    = D['P1'][Nf]



### Example

Notation

$P_{NS}^+$ == PNSP

$P_{NS}^-$ == PNSM

$P_{NS}^v$ == PNSV

$P_{ij}$ == P$[i,j]$



In [15]:
conf={}
conf['mellin']=MELLIN(8,c=1.9)
kernels=KERNELS()

Nf   = 3
iord = 0 # 0: LO, 1: NLO

print('PNSP=',kernels.PNSP[Nf,iord][:5])  
print('PNSM=',kernels.PNSM[Nf,iord][:5])
print('PNSV=',kernels.PNSV[Nf,iord][:5])  
print('Pqq=',kernels.P[Nf,iord][0,0][:5])
print('Pqg=',kernels.P[Nf,iord][0,1][:5])
print('Pgq=',kernels.P[Nf,iord][1,0][:5])
print('Pgg=',kernels.P[Nf,iord][1,1][:5])


PNSP= [-3.29751227-0.003672j   -3.28239399-0.01886268j -3.25734425-0.04425009j
 -3.22574645-0.07666977j -3.19187779-0.11192402j]
PNSM= [-3.29751227-0.003672j   -3.28239399-0.01886268j -3.25734425-0.04425009j
 -3.22574645-0.07666977j -3.19187779-0.11192402j]
PNSV= [-3.29751227-0.003672j   -3.28239399-0.01886268j -3.25734425-0.04425009j
 -3.22574645-0.07666977j -3.19187779-0.11192402j]
Pqq= [-3.29751227-0.003672j   -3.28239399-0.01886268j -3.25734425-0.04425009j
 -3.22574645-0.07666977j -3.19187779-0.11192402j]
Pqg= [2.09832462-0.00144015j 2.10424893-0.00742313j 2.11406325-0.01751226j
 2.12643636-0.03055833j 2.13968454-0.04494893j]
Pgq= [4.04606377-0.00764145j 4.07743597-0.0396861j  4.12936105-0.09480304j
 4.19464249-0.16804641j 4.26414584-0.25135344j]
Pgg= [-0.41408374-0.02401511j -0.31540448-0.12431162j -0.15202544-0.29535728j
  0.05357974-0.52005308j  0.27291856-0.77242535j]


### Comments

- The arrays printed above are spliting functins in Mellin space evaluated at a given contour specified in the class Mellin 

- They are given as arrys for $N_f=3,4,5$ and `iord=0,1`. 

- The splitting kernels obays certain important properties: 
  + **valence number sum rule** 
  + **momentum sum rule**


### Exercise 4.A (time: 5 mins)

- Set N=1 in the mellin class via   `conf['mellin'].N=np.array([1])`  
- check the valence number sum rule at LO
- **Hint**: $\frac{\partial q_v}{\partial \ln \mu^2}(N)=P_{NS}^-(N)q_v(N)=0$ for $N=1$

### Exercise 4.B (time: 5 mins)

- Set N=2 in the mellin class via   `conf['mellin'].N=np.array([2])`  
- check the momentum sum rule at LO
- **Hint**: $\frac{\partial (\Sigma+G)}{\partial \ln \mu^2}(N)
            =(P_{qq}+P_{gq})\Sigma+(P_{qg}+P_{gg})G=0$ for $N=2$ 

## DGLAP evolution

In [16]:
class DGLAP:
 
    def __init__(self):
        """
        Nomenclature:
        BC = bondary condition
        EO = evolution operator
        BM = beta matrix
        """
        self.mell  = conf['mellin']
        self.asevo = conf['alphaS']
        self.spl   = conf['kernels']
        self.order = conf['order']
        self.get_beta_matrix()
        self.get_evolution_operators()

    def get_beta_matrix(self):
        """
        beta matrix stores the beta coeffs for 
        different Nf and perturbative orders
        """
        beta=self.asevo.beta
        nflav =6+1 # up to 6 flavors
        norder=2+1 # up to NNLO
        b0=np.zeros(nflav)  
        b=np.zeros((nflav,norder))
        for Nf in range(3,nflav):
            b0[Nf]=beta[Nf][0]
            for order in range(norder): 
                b[Nf,order]=beta[Nf,order]/beta[Nf,0]
        # globalize
        self.b=b
        self.nflav=nflav
        self.norder=norder

    def get_non_singlet_evolution_operator(self,P):

        b      = self.b
        nflav  = self.nflav
        norder = self.norder
        Nsize  = self.mell.N.size
        beta   = self.asevo.beta

        # construct evolution operators
        R=np.zeros((nflav,norder,Nsize),dtype=complex)
        U=np.zeros((nflav,norder,Nsize),dtype=complex)
        RT=np.zeros((nflav,norder,Nsize),dtype=complex)

        # LO operators
        for Nf in range(3,nflav):
            R[Nf,0]=P[Nf,0]/beta[Nf,0]
            U[Nf,0]=np.ones(Nsize)

        # NLO operators
        for Nf in range(3,nflav):
            R[Nf,1]=P[Nf,1]/beta[Nf,0]-b[Nf,1]*R[Nf,0]
            RT[Nf,1]=R[Nf,1]*U[Nf,0]
            U[Nf,1]=-RT[Nf,1]/1.0

        return {'U':U,'R':R}

    def get_singlet_evolution_operator(self,P):

        b      = self.b
        nflav  = self.nflav
        norder = self.norder
        Nsize  = self.mell.N.size
        beta   = self.asevo.beta

        # construct evolution operators
        R=np.zeros((nflav,norder,2,2,Nsize),dtype=complex)
        U=np.zeros((nflav,norder,2,2,Nsize),dtype=complex)
        RT=np.zeros((nflav,norder,2,2,Nsize),dtype=complex)
        RP=np.zeros((nflav,2,2,Nsize),dtype=complex)
        RM=np.zeros((nflav,2,2,Nsize),dtype=complex)
        rp=np.zeros((nflav,Nsize),dtype=complex)
        rm=np.zeros((nflav,Nsize),dtype=complex)

        # LO operators
        for Nf in range(3,nflav):
            R[Nf,0]=P[Nf,0]/beta[Nf,0]
            r1=R[Nf,0,0,0]+R[Nf,0,1,1]
            r2=((R[Nf,0,0,0]-R[Nf,0,1,1])**2+4*R[Nf,0,0,1]*R[Nf,0,1,0])**0.5
            rp[Nf]=0.5*(r1+r2)
            rm[Nf]=0.5*(r1-r2)

            RP[Nf,0,0]=(R[Nf,0,0,0]-rm[Nf])/(rp[Nf]-rm[Nf])
            RP[Nf,1,1]=(R[Nf,0,1,1]-rm[Nf])/(rp[Nf]-rm[Nf])
            RP[Nf,0,1]=R[Nf,0,0,1]/(rp[Nf]-rm[Nf])
            RP[Nf,1,0]=R[Nf,0,1,0]/(rp[Nf]-rm[Nf])

            RM[Nf,0,0]=1-RP[Nf,0,0]
            RM[Nf,1,1]=1-RP[Nf,1,1]
            RM[Nf,0,1]= -RP[Nf,0,1]
            RM[Nf,1,0]= -RP[Nf,1,0]

            U[Nf,0,0,0]=np.zeros(Nsize)+1
            U[Nf,0,0,1]=np.zeros(Nsize)
            U[Nf,0,1,0]=np.zeros(Nsize)
            U[Nf,0,1,1]=np.zeros(Nsize)+1

        # NLO operators
        for Nf in range(3,nflav):
            R[Nf,1]=P[Nf,1]/beta[Nf,0]-b[Nf,1]*R[Nf,0]
            RT[Nf,1]=R[Nf,1]
            U11=-np.einsum('ij...,jk...,kl...->il...',RP[Nf],RT[Nf,1],RP[Nf])
            U12=-np.einsum('ij...,jk...,kl...->il...',RM[Nf],RT[Nf,1],RM[Nf])
            U13= np.einsum('ij...,jk...,kl...->il...',RP[Nf],RT[Nf,1],RM[Nf])/(rm[Nf]-rp[Nf]-1)
            U14= np.einsum('ij...,jk...,kl...->il...',RM[Nf],RT[Nf,1],RP[Nf])/(rp[Nf]-rm[Nf]-1)
            U[Nf,1]=U11+U12+U13+U14

        return {'U':U,'rp':rp,'rm':rm,'RP':RP,'RM':RM}

    def get_evolution_operators(self):
        spl=self.spl
        self.EO_NSP = self.get_non_singlet_evolution_operator(spl.PNSP)
        self.EO_NSM = self.get_non_singlet_evolution_operator(spl.PNSM)
        self.EO_NSV = self.get_non_singlet_evolution_operator(spl.PNSV)
        self.EO_S   = self.get_singlet_evolution_operator(spl.P)

    def evolve_nonsinglet(self,EO,qini,Q2ini,Q2fin,Nf):
        a = self.asevo.get_a(Q2fin)
        a0= self.asevo.get_a(Q2ini)
        L=np.power(a/a0,-EO['R'][Nf,0])
        operator=np.copy(L)
        if  self.order>=1: 
            operator+= (a-a0)*EO['U'][Nf,1]*L
        q=operator*qini
        return q

    def evolve_singlet(self,EO,qini,Q2ini,Q2fin,Nf):
        Nsize=self.mell.N.size
        a = self.asevo.get_a(Q2fin)
        a0= self.asevo.get_a(Q2ini)
        L=np.power(a/a0,-EO['rp'][Nf])*EO['RP'][Nf]+np.power(a/a0,-EO['rm'][Nf])*EO['RM'][Nf]
        operator = np.copy(L)
        if  self.order>=1: 
            operator+= a*np.einsum('ij...,jk...->ik...',EO['U'][Nf,1],L,dtype=complex)\
                      -a0*np.einsum('ij...,jk...->ik...',L,EO['U'][Nf,1],dtype=complex)
        q=np.einsum('ij...,j...->i...',operator,qini,dtype=complex)
        return q

    def evolve(self,BC,Q2ini,Q2fin,Nf):

        nflav=self.nflav
        Nsize=self.mell.N.size

        EO_NSP =  self.EO_NSP 
        EO_NSM =  self.EO_NSM 
        EO_NSV =  self.EO_NSV 
        EO_S   =  self.EO_S   

        # retrieve boundary condition
        vm,vp={},{}
        for k in [i**2-1 for i in range(1,nflav)]:
            vm[k]=np.copy(BC['vm'][k]) 
            vp[k]=np.copy(BC['vp'][k]) 
        qv=np.copy(BC['qv']) 
        q=np.copy(BC['q']) 

        # evolve
        qv=self.evolve_nonsinglet(EO_NSV,qv,Q2ini,Q2fin,Nf)
        q=self.evolve_singlet(EO_S,q,Q2ini,Q2fin,Nf)
        qs = q[0]

        if   Nf==3:
            vm[3] =self.evolve_nonsinglet(EO_NSM,vm[3],Q2ini,Q2fin,Nf)
            vm[8] =self.evolve_nonsinglet(EO_NSM,vm[8],Q2ini,Q2fin,Nf)
            vm[15]=qv
            vm[24]=qv
            vm[35]=qv
            vp[3] =self.evolve_nonsinglet(EO_NSP,vp[3],Q2ini,Q2fin,Nf)
            vp[8] =self.evolve_nonsinglet(EO_NSP,vp[8],Q2ini,Q2fin,Nf)
            vp[15]=qs
            vp[24]=qs
            vp[35]=qs

        elif Nf==4:
            vm[3] =self.evolve_nonsinglet(EO_NSM,vm[3],Q2ini,Q2fin,Nf)
            vm[8] =self.evolve_nonsinglet(EO_NSM,vm[8],Q2ini,Q2fin,Nf)
            vm[15]=self.evolve_nonsinglet(EO_NSM,vm[15],Q2ini,Q2fin,Nf)
            vm[24]=qv
            vm[35]=qv
            vp[3] =self.evolve_nonsinglet(EO_NSP,vp[3],Q2ini,Q2fin,Nf)
            vp[8] =self.evolve_nonsinglet(EO_NSP,vp[8],Q2ini,Q2fin,Nf)
            vp[15]=self.evolve_nonsinglet(EO_NSP,vp[15],Q2ini,Q2fin,Nf)
            vp[24]=qs
            vp[35]=qs

        elif Nf==5:
            vm[3] =self.evolve_nonsinglet(EO_NSM,vm[3],Q2ini,Q2fin,Nf)
            vm[8] =self.evolve_nonsinglet(EO_NSM,vm[8],Q2ini,Q2fin,Nf)
            vm[15]=self.evolve_nonsinglet(EO_NSM,vm[15],Q2ini,Q2fin,Nf)
            vm[24]=self.evolve_nonsinglet(EO_NSM,vm[24],Q2ini,Q2fin,Nf)
            vm[35]=qv
            vp[3] =self.evolve_nonsinglet(EO_NSP,vp[3],Q2ini,Q2fin,Nf)
            vp[8] =self.evolve_nonsinglet(EO_NSP,vp[8],Q2ini,Q2fin,Nf)
            vp[15]=self.evolve_nonsinglet(EO_NSP,vp[15],Q2ini,Q2fin,Nf)
            vp[24]=self.evolve_nonsinglet(EO_NSP,vp[24],Q2ini,Q2fin,Nf)
            vp[35]=qs

        elif Nf==6:
            vm[3] =self.evolve_nonsinglet(EO_NSM,vm[3],Q2ini,Q2fin,Nf)
            vm[8] =self.evolve_nonsinglet(EO_NSM,vm[8],Q2ini,Q2fin,Nf)
            vm[15]=self.evolve_nonsinglet(EO_NSM,vm[15],Q2ini,Q2fin,Nf)
            vm[24]=self.evolve_nonsinglet(EO_NSM,vm[24],Q2ini,Q2fin,Nf)
            vm[35]=self.evolve_nonsinglet(EO_NSM,vm[35],Q2ini,Q2fin,Nf)
            vp[3] =self.evolve_nonsinglet(EO_NSP,vp[3],Q2ini,Q2fin,Nf)
            vp[8] =self.evolve_nonsinglet(EO_NSP,vp[8],Q2ini,Q2fin,Nf)
            vp[15]=self.evolve_nonsinglet(EO_NSP,vp[15],Q2ini,Q2fin,Nf)
            vp[24]=self.evolve_nonsinglet(EO_NSP,vp[24],Q2ini,Q2fin,Nf)
            vp[35]=self.evolve_nonsinglet(EO_NSP,vp[35],Q2ini,Q2fin,Nf)

        # flav decomposition
        qp =np.zeros((nflav,Nsize),dtype=complex)
        qm =np.zeros((nflav,Nsize),dtype=complex)

        qm[6]=(qv-vm[35])/6.
        qm[5]=qm[6]+(vm[35]-vm[24])/5.
        qm[4]=qm[5]+(vm[24]-vm[15])/4.
        qm[3]=qm[4]+(vm[15]-vm[8])/3.
        qm[2]=qm[3]+(vm[8]-vm[3])/2.
        qm[1]=qm[3]+(vm[8]+vm[3])/2.

        qp[6]=(qs-vp[35])/6.
        qp[5]=qp[6]+(vp[35]-vp[24])/5.
        qp[4]=qp[5]+(vp[24]-vp[15])/4.
        qp[3]=qp[4]+(vp[15]-vp[8])/3.
        qp[2]=qp[3]+(vp[8]-vp[3])/2.
        qp[1]=qp[3]+(vp[8]+vp[3])/2.

        # for internal usage 
        result={}
        result['vm']=vm 
        result['vp']=vp 
        result['qv']=qv 
        result['q'] =q 


        # for external usage
        result['up']=qp[1]
        result['um']=qm[1]
        result['dp']=qp[2]
        result['dm']=qm[2]
        result['sp']=qp[3]
        result['sm']=qm[3]
        result['cp']=qp[4]
        result['cm']=qm[4]
        result['bp']=qp[5]
        result['bm']=qm[5]
        result['tp']=qp[6]
        result['tm']=qm[6]

        result['qp']=qp
        result['qm']=qm

        result['g'] =q[1]
        result['sigma'] =q[0]
        result['u']=0.5*(qp[1]+qm[1])
        result['d']=0.5*(qp[2]+qm[2])
        result['s']=0.5*(qp[3]+qm[3])
        result['c']=0.5*(qp[4]+qm[4])
        result['b']=0.5*(qp[5]+qm[5])
        result['ub']=0.5*(qp[1]-qm[1])
        result['db']=0.5*(qp[2]-qm[2])
        result['sb']=0.5*(qp[3]-qm[3])
        result['cb']=0.5*(qp[4]-qm[4])
        result['bb']=0.5*(qp[5]-qm[5])

        return result


### Comments


- The DGLAP evolution takes place at
 + `evolve_nonsinglet(EO,q,Q2ini,Q2fin,Nf)`
 + `evolve_singlet(EO,q,Q2ini,Q2fin,Nf)`
- These functions takes pdfs moments (`q`) at `Q2ini` and evolve them upto `Q2fin` using `Nf` active flavors  
- The most relevant function to pay attention to is `evolve(BC,Q2ini,Q2fin,Nf)`. This functions orchestrates the full evoltion (singlet and non-singlet) of pdfs defined as bondary conditions `BC` at `Q2ini` 
- The output of `evolve(BC,Q2ini,Q2fin,Nf)` contains different flavors of flavors combinations such as $\Sigma=u^++d^++s^++c^++b^+$ with $q^+=q+\bar{q}$
- Here is an example for how to write `BC`

In [17]:
conf={}
conf['order']   = 1 #-- LL=0  NLL=1
conf['aux']     = AUX()
conf['Q20']     = 1.27**2
conf['mellin']  = MELLIN(npts=4)
conf['kernels'] = KERNELS()
conf['alphaS']  = ALPHAS()
dglap           = DGLAP()

In [18]:
N=conf['mellin'].N
zero=np.zeros(N.size,dtype=complex)
ones=np.ones(N.size,dtype=complex)

a=-0.5; b=6.0
pdf=gamma(N+a)*gamma(b+1)/gamma(N+a+b+1)  #--toy pdf at input scale

#-- lets define toy pdf flavors with different normalizations 

g  = 1*pdf
up = 2*pdf  # u+ub
dp = 3*pdf  # d+db
sp = 4*pdf  # s+sb 
cp = zero
bp = zero
tp = zero

um = 5*pdf # u-ub
dm = 6*pdf # d-db
sm = zero
cm = zero
bm = zero
tm = zero

# flav composition
vm,vp={},{}
vm[35]= bm + cm + dm + sm - 5*tm + um
vm[24]= -4*bm + cm + dm + sm + um 
vm[15]= -3*cm + dm + sm + um
vm[8] = dm - 2*sp + 2*(-sm + sp) + um
vm[3] = -dm + um
vm[0] = np.zeros(N.size,dtype=complex)
vp[0] = np.zeros(N.size,dtype=complex)
vp[3] = -dp + up
vp[8] = dp - 2*sp + up 
vp[15]= -3*cp + dp + sp + up 
vp[24]= -4*bp + cp + dp + sp + up 
vp[35]= bp + cp + dp + sp - 5*tp + up 
qs    = bp + cp + dp + sp + tp + up 
qv    = bm + cm + dm + sm + tm + um 
q     = np.zeros((2,N.size),dtype=complex)
q[0]=np.copy(qs)
q[1]=np.copy(g)

BC={}
BC['vm']=vm 
BC['vp']=vp 
BC['qv']=qv
BC['q'] =q


- What actually evolvs in the code is the flavor combinations $v^{\pm}_i,~q_v,~q$
- Once $v^{\pm}_i,~q_v,~q$ are evolved we can transform them into $g,u,d,s,...$
- **attention**: $q$ has 2 components and evolves as a matrix
- Lets evolve $v^{\pm}_i,~q_v,~q$ that we just built in `BC`

In [19]:
Nf=4 
Q2ini=conf['aux'].mc2  
Q2fin=conf['aux'].mb2
output = dglap.evolve(BC,Q2ini,Q2fin,Nf)

In [20]:
output.keys()

dict_keys(['vm', 'vp', 'qv', 'q', 'up', 'um', 'dp', 'dm', 'sp', 'sm', 'cp', 'cm', 'bp', 'bm', 'tp', 'tm', 'qp', 'qm', 'g', 'sigma', 'u', 'd', 's', 'c', 'b', 'ub', 'db', 'sb', 'cb', 'bb'])

In [21]:
BC['vm'][3][:5]

array([-0.05659416+0.00059278j, -0.05878718+0.00295196j,
       -0.06162245+0.00636407j, -0.06375713+0.00924953j,
       -0.06542741+0.0117328j ])

In [22]:
output['vm'][3][:5]

array([-0.04539473+0.00051295j, -0.04729027+0.0025628j ,
       -0.04973765+0.00554789j, -0.0515753 +0.00808787j,
       -0.05300826+0.01028344j])

The above shows how $v^-_3= u_v-d_v$ evolves along the mellin contour 

### Exercise 5.A (time: 5 mins)


- A physical proton pdfs will need $\int_0^1~dx~u_v(x)=2$ and $\int_0^1~dx~d_v(x)=1$ 
- Modify the ``BC`` we use above and show that the valence number sum rules don't evolve
- **hint**: 
  + Set N=1 in the mellin class via   `conf['mellin'].N=np.array([1])`  
  + Set `um = np.array([2])`
  + Set `dm = np.array([1])`
  + Printe the output values `um` and `dm`



### Exercise 5.B (time: 5 mins)


- A physical proton pdfs will need $\int_0^1dx~x~\left[\Sigma(x) + g(x)\right]=1$ 
- Modify the ``BC`` we use above and show that the momentum sum rules don't evolve
- **hint**: 
  + Set N=2 in the mellin class via   `conf['mellin'].N=np.array([2])`  
  + Set `g= 1-up-dp-sp`
  + Print the values of `output['g']+output['sigma'])`

## The PDFs class

In [23]:
class PDF: 

    def __init__(self):

        self.mu20=conf['mu20']
        self.mc2=conf['aux'].mc2
        self.mb2=conf['aux'].mb2
      
        conf['kernels']= KERNELS()
        self.dglap     = DGLAP()
        self.mellin    = conf['mellin']

        self.set_params()
        self.setup()

    def set_params(self):
        """
        f(x) = norm * x**a * (1-x)**b * (1+c*x**0.5+d*x)
        """
        params={}
        params['g']    =np.array([0.09,-0.17,9.97,-4.055,4.92])
        params['uv']   =np.array([0.324, -0.5,3.54,2.8,5.8])
        params['dv']   =np.array([0.28, -0.12,5.5,-2.47,4.75])
        params['sea1'] =np.array([0.001,-1.22,10.0,0,0])
        params['sea2'] =np.array([0.001,-1.22,10.0,0,0])
        params['db']   =np.array([0.0025,-0.12,10.0,-3.2,-0.0095])
        params['ub']   =np.array([0.00176,0.22,10.0,-3.77,4.3])
        params['s']    =np.array([ 0.02117505,-0.7834729,5.94912366,0.,0.])
        params['sb']   =np.array([ 0.03185304,-0.4761831,10.,0.,0.])
        self.params=params
        self.FLAV = ['g','uv','dv','sea1','sea2','db','ub','s','sb']
        self.PAR  = ['N','a','b','c','d']


    def set_sumrules(self):
        #--valence
        self.params['uv'][0]=1    
        self.params['uv'][0]=2/self.get_moments('uv',1)

        self.params['dv'][0]=1    
        self.params['dv'][0]=1/self.get_moments('dv',1)

        #--strange
        self.params['s'][0]=1    
        self.params['s'][0]=self.get_moments('sb',1)/self.get_moments('s',1)
    
        #--msr
        sea1=self.get_moments('sea1',2)
        sea2=self.get_moments('sea2',2)
        up=self.get_moments('uv',2)+2*(sea1+self.get_moments('ub',2))
        dp=self.get_moments('dv',2)+2*(sea1+self.get_moments('db',2))
        sp=(sea2+self.get_moments('s',2))+(sea2+self.get_moments('sb',2))
        self.params['g'][0]=1    
        self.params['g'][0]=(1-up-dp-sp)/self.get_moments('g',2)
        g=self.get_moments('g',2)
        msr=g+up+dp+sp

        #--share
        self.sr={}
        self.sr['msr']      = msr
        self.sr['uv(1)']    = self.get_moments('uv',1)
        self.sr['dv(1)']    = self.get_moments('dv',1)
        self.sr['s-sb(1)']  = self.get_moments('s',1)-self.get_moments('sb',1)
        self.sr['s-sb(2)']  = self.get_moments('s',2)-self.get_moments('sb',2)
        self.sr['db-ub(1)'] = self.get_moments('db',1)-self.get_moments('ub',1)
        self.sr['db-ub(2)'] = self.get_moments('db',2)-self.get_moments('ub',2)

    def set_moms(self):

        sea1=self.get_moments('sea1')
        sea2=self.get_moments('sea2')

        moms={}
        moms['g']  = self.get_moments('g')
        moms['up'] = self.get_moments('uv')+2*(sea1+self.get_moments('ub'))
        moms['dp'] = self.get_moments('dv')+2*(sea1+self.get_moments('db'))
        moms['sp'] = 2*sea2+self.get_moments('s')+self.get_moments('sb')
        moms['um'] = self.get_moments('uv')
        moms['dm'] = self.get_moments('dv')
        moms['sm'] = self.get_moments('s')-self.get_moments('sb')
        self.moms0=moms
        self.get_BC(moms)
        
    def setup(self):
        self.set_sumrules()
        self.set_moms()
        #--store moments of a given Q2 that has been already calculated
        self.storage={}

    def beta(self,a,b):
        return gamma(a)*gamma(b)/gamma(a+b)

    def get_moments(self,flav,N=None):
        """
        if N==None: then parametrization is to be use to compute moments along mellin contour
        else the Nth moment is returned
        """
        if N==None: N=self.mellin.N
        M,a,b,c,d=self.params[flav]
        mom=self.beta(N+a,b+1)+c*self.beta(N+a+0.5,b+1)+d*self.beta(N+a+1,b+1)
        norm=self.beta(2+a,b+1)+c*self.beta(2+a+0.5,b+1)+d*self.beta(2+a+1,b+1)
        return M*mom/norm

    def _get_BC(self,g,up,um,dp,dm,sp,sm,cp,cm,bp,bm,tp,tm):
        N=self.mellin.N

        # flav composition
        vm,vp={},{}
        vm[35]= bm + cm + dm + sm - 5*tm + um
        vm[24]= -4*bm + cm + dm + sm + um 
        vm[15]= -3*cm + dm + sm + um
        vm[8] = dm - 2*sp + 2*(-sm + sp) + um
        vm[3] = -dm + um
        vm[0] = np.zeros(N.size,dtype=complex)
        vp[0] = np.zeros(N.size,dtype=complex)
        vp[3] = -dp + up
        vp[8] = dp - 2*sp + up 
        vp[15]= -3*cp + dp + sp + up 
        vp[24]= -4*bp + cp + dp + sp + up 
        vp[35]= bp + cp + dp + sp - 5*tp + up 
        qs    = bp + cp + dp + sp + tp + up 
        qv    = bm + cm + dm + sm + tm + um 
        q     = np.zeros((2,N.size),dtype=complex)
        q[0]=np.copy(qs)
        q[1]=np.copy(g)

        BC={}
        BC['vm']=vm 
        BC['vp']=vp 
        BC['qv']=qv
        BC['q'] =q
        return BC
  
    def get_BC(self,moms):

        N=self.mellin.N
        zero=np.zeros(N.size,dtype=complex)

        #--BC for Nf=3
        g   = moms['g']
        up  = moms['up']
        um  = moms['um']
        dp  = moms['dp']
        dm  = moms['dm']
        sp  = moms['sp']
        sm  = moms['sm']
        cp  = zero
        cm  = zero
        bp  = zero
        bm  = zero
        self.BC3=self._get_BC(g,up,um,dp,dm,sp,sm,zero,zero,zero,zero,zero,zero)

        #--BC for Nf=4
        BC4=self.dglap.evolve(self.BC3,self.mu20,self.mc2,3)
        g =BC4['g']
        up=BC4['up']
        dp=BC4['dp']
        sp=BC4['sp']
        cp=BC4['cp']
        bp=BC4['bp']
        tp=BC4['tp']
        um=BC4['um']
        dm=BC4['dm']
        sm=BC4['sm']
        cm=BC4['cm']
        bm=BC4['bm']
        tm=BC4['tm']
        self.BC4=self._get_BC(g,up,um,dp,dm,sp,sm,cp,cm,bp,bm,tp,tm)

        #--BC for Nf=5
        BC5=self.dglap.evolve(self.BC4,self.mc2,self.mb2,4)
        g =BC5['g']
        up=BC5['up']
        dp=BC5['dp']
        sp=BC5['sp']
        cp=BC5['cp']
        bp=BC5['bp']
        tp=BC5['tp']
        um=BC5['um']
        dm=BC5['dm']
        sm=BC5['sm']
        cm=BC5['cm']
        bm=BC5['bm']
        tm=BC5['tm']
        self.BC5=self._get_BC(g,up,um,dp,dm,sp,sm,cp,cm,bp,bm,tp,tm)

    def evolve(self,mu2):
        if mu2 not in self.storage:
            if self.mb2<mu2: 
                self.storage[mu2]=self.dglap.evolve(self.BC5,self.mb2,mu2,5)
            elif self.mc2<=mu2 and mu2<=self.mb2: 
                self.storage[mu2]=self.dglap.evolve(self.BC4,self.mc2,mu2,4)
            elif mu2<self.mc2: 
                self.storage[mu2]=self.dglap.evolve(self.BC3,self.mu20,mu2,3)

    def get_xF(self,x,mu2,flav,evolve=True):
        if evolve: self.evolve(mu2)
        return x*conf['mellin'].invert(x,self.storage[mu2][flav])


### Comments:

- The class `PDF` builds upon various parts we have introduced earlier. 
- The are 3 parts of this class that are important  to discuss: 
  1. parametrization, 
  2. sum rules 
  3. Boundary conditions   



#### 1) Parametrization 

We use the following generic tempate function of $x$ with **free** parameters $M,a,b,c,d$

$
T(x;p) = M ~ \frac{x^a  (1-x)^b ~ (1+c \sqrt{x}+d x)}
{\int_0^1~dy~y y^a  (1-y)^b ~ (1+c \sqrt{y}+d y)}
$

Where $p=(M,a,b,c,d)$.  The Mellin transform of the shape function is given by 

$
{\cal M}[T] = M~\frac{B(N+a,b+1)+cB(N+a+1/2,b+1)+dB(N+a+1,b+1)}
                     {B(2+a,b+1)+cB(2+a+1/2,b+1)+dB(2+a+1,b+1)}
$

See the function  ``get_moments(flav)``.
The basic shapes we build with the template function $T(x;p)$ are 

- $u_v=T(x;p_{u_v})$
- $d_v=T(x;p_{d_v})$
- $\bar{u}_v=T(x;p_{\bar{u}})$
- $\bar{d}_v=T(x;p_{\bar{d}})$
- $s_v=T(x;p_{s})$
- $\bar{s}_v=T(x;p_{\bar{s}})$
- $S_{1,2}=T(x;p_{S})$
- $g=T(x;p_{g})$
 
With these basic shapes we build the pdfs as

- $\bar{u}=\bar{u}_v+S_1$
- $\bar{d}=\bar{d}_v+S_1$
- $s=s_v+S_2$
- $\bar{s}=\bar{s}_v+S_2$
- $u = u_v+\bar{u}$
- $d = d_v+\bar{d}$

The modeling of input scale pdfs is designed to satisfy the following conditions

- $\lim_{x\to 0} u=\bar{u}=\bar{d}=d=s=\bar{s} = S$
- $\lim_{x\to 0} u_v,d_v,s_v,\bar{s}_v = 0$

#### 2) Sum rules 

The shape parameter $M$ can be fixed in order to satisfy the sumrules. Lets write the template function in terms of $\tilde{T}$ to factor out the $M$ factor 

$
T(x;p) = M~\tilde{T}(x;\tilde{p}) 
$

$
\tilde{T}(x;\tilde{p})=\frac{x^a  (1-x)^b ~ (1+c \sqrt{x}+d x)}
{\int_0^1~dy~y y^a  (1-y)^b ~ (1+c \sqrt{y}+d y)}
$

Then the $M$ parameters can be set by 

- $\int_0^1~dx~u_v=2$  $\to$  $M_{u_v}=\frac{2}{\int_0^1~dx~\tilde{u}_v}$
- $\int_0^1~dx~d_v=1$  $\to$  $M_{d_v}=\frac{1}{\int_0^1~dx~\tilde{d}_v}$
- $\int_0^1~dx~(s-\bar{s})=0$ $\to$  $M_{\bar{s}}=\frac{\int_0^1~dx~s}{\int_0^1~dx~\tilde{\bar{s}}}$
- $\int_0^1~dx~x~(g+\Sigma)=1$ $\to$ $M_g=\frac{1-\int_0^1~dx~x~\Sigma}{\int_0^1~dx~x~\tilde{g}}$

#### 3) Boundary conditions

- Just like the RGE for $\alpha_S$, the DGLAP splitting kernels are discontinuos at the mass thresholds. 
- Because of this, we need to build we need to build pdf boundary conditions for $N_f=3,4,5$
- The function `get_BC(moms)` show how to build boudary conditions `BC3,BC4,BC5` starting from input scale pdfs in mellin space `moms`

### Example

In [24]:
conf={}
conf['order']    = 1 #-- LL=0  NLL=1
conf['aux']      = AUX()
conf['mu20']     = 1.27**2  #--this is the input scale
conf['mellin']   = MELLIN(npts=8)
conf['alphaS']   = ALPHAS()
pdf= PDF()        

In [25]:
x=0.1; Q2=10.0
for flav in ['g','u','d','s','ub','db','sb']:
    print('x%s='%flav,pdf.get_xF(x,Q2,flav))

xg= 1.5477859845138409
xu= 0.5856159878392141
xd= 0.32399164203459674
xs= 0.09212608647680302
xub= 0.029775624387608064
xdb= 0.028654226776980252
xsb= 0.14628466776419294


### Exercise 6 (time: 5 mins)

- Check the valence number and momentum sum rules at $\mu^2=m_c^2,10,100,1000$


## DIS theory at NLO

In [26]:
class THEORY:
  
    def __init__(self):

        self.mellin  = conf['mellin']
        self.setup()
        self.order=conf['order']
            
        self.mc2  = conf['aux'].mc**2
        self.mb2  = conf['aux'].mb**2
        self.M2=conf['aux'].M2
 

    def setup(self):
        #--abbreviations
        zeta2=fp.zeta(2)
        zeta3=fp.zeta(3)
        N   = self.mellin.N 
        NP1 = N + 1
        NP2 = N + 2
        NM1 = N - 1
        NS=N*N
        psi=lambda i,N: fp.psi(i,complex(N.real,N.imag))
        S1 = np.array([fp.euler  + psi(0,n+1) for n in N])
        S2 = np.array([zeta2- psi(1,n+1) for n in N])
        S1S=S1**2

        #--hard coeffs
        orders=2

        self.C2Q = np.zeros((orders,N.size),dtype=complex)
        self.C2G = np.zeros((orders,N.size),dtype=complex)
        self.CLQ = np.zeros((orders,N.size),dtype=complex)
        self.CLG = np.zeros((orders,N.size),dtype=complex)
        self.C3Q = np.zeros((orders,N.size),dtype=complex)

        self.C2Q[0]  = np.ones(N.size)  
        self.C3Q[0]  = np.ones(N.size)  

        CF=conf['aux'].CF
        TR=conf['aux'].TR
        
        #--Nucl. Phys. B192 (1981) 417
        self.C2Q[1]  = CF*(2.0*S1S - 2.0*S2 + 3.0*S1 - 2.0*S1/N/NP1 + 3.0/N + 4.0/NP1 + 2.0/NS - 9.0)
        self.C2G[1]  = -2*TR*(S1*(NS + N + 2.0)/N/NP1/NP2 + 1.0/N - 1.0/NS - 6.0/NP1 + 6.0/NP2)  
        self.CLQ[1]  = CF*4/NP1  
        self.CLG[1]  = 8.0*TR/NP1/NP2
        self.C3Q[1]  = CF*(2.0*S1S - 2.0*S2 + 3.0*S1 - 2.0*S1/N/NP1 + 1.0/N + 2.0/NP1 + 2.0/NS - 9.0)
    
    def get_aX(self,aX,i,Q2):
        """
        couplings 
        {q} = {u,d,s,c,b} 
        """

        alpha = conf['eweak'].get_alpha(Q2)
        sin2w = conf['eweak'].get_sin2w(Q2)
        KQ2 = conf['aux'].GF*conf['aux'].mZ2/(2*2**0.5*np.pi*alpha)* Q2/(Q2+conf['aux'].mZ2)

        Ve=-0.5+2*sin2w
        Ae=-0.5
        Ve2pAe2=Ve*Ve+Ae*Ae

        eu=2.0/3.0
        Vu=0.5-2*eu*sin2w
        Au=0.5
        Vu2pAu2=Vu*Vu+Au*Au

        ed=-1.0/3.0
        Vd=-0.5-2*ed*sin2w
        Ad=-0.5
        Vd2pAd2=Vd*Vd+Ad*Ad

        #--for individual channels, conventions are taken from PDG
        #--pdg.lbl.gov/2019/reviews/rpp2019-rev-structure-functions.pdf
        #--see Eq. (18.18)
        if aX=='ap':
            if i==1 or i==4:
                return eu*eu - 2*eu*Ve*Vu*KQ2 + Ve2pAe2*Vu2pAu2*KQ2**2
            elif i==2 or i==3 or i==5:
                return ed*ed - 2*ed*Ve*Vd*KQ2 + Ve2pAe2*Vd2pAd2*KQ2**2
        if aX=='am':
            if i==1 or i==4:
                return -2*eu*Ae*Au*KQ2 + 4*Ve*Ae*Vu*Au*KQ2**2
            elif i==2 or i==3 or i==5:
                return -2*ed*Ae*Ad*KQ2 + 4*Ve*Ae*Vd*Ad*KQ2**2

        
    #--twist 2 unpolarized structure functions 
  
    def get_T2CFX(self,stf,nucleon,Q2):   
        """
        CF(2,L,3) = F2/x,FL/x,F3  
        """ 
        conf['pdf'].evolve(Q2)

        g =conf['pdf'].storage[Q2]['g']    
        Nf=conf['alphaS'].get_Nf(Q2) 
        a=conf['alphaS'].get_a(Q2) 

        if stf=='F2':
            CQ = self.C2Q[0] + a*self.order*self.C2Q[1] 
            CG = self.C2G[0] + a*self.order*self.C2G[1]
            q=np.copy(conf['pdf'].storage[Q2]['qp'])   
            aX='ap'
 
        elif stf=='FL':
            CQ = a*self.order*self.CLQ[1]
            CG = a*self.order*self.CLG[1]
            q=np.copy(conf['pdf'].storage[Q2]['qp'])
            aX='ap'
  
        elif stf=='F3':
            CQ = self.C3Q[0] + a*self.order*self.C3Q[1]
            CG = 0
            q=np.copy(conf['pdf'].storage[Q2]['qm']) 
            aX='am'
  
        if nucleon=="n":
            qup=np.copy(q[1])
            qdn=np.copy(q[2])
            q[1]=qdn
            q[2]=qup
  
        FX  = np.zeros(self.mellin.N.size,dtype=complex) 
        for i in range(1,Nf+1):
            aXval = self.get_aX(aX,i,Q2)
            FX+=aXval*(CQ*q[i] + 2*CG*g)
  
        return FX

    def get_T2CWX(self,stf,nucleon,Q2,sign):  
        """
        CF(2,L,3) = W2/x,WL/x,W3  
        """ 
        conf['pdf'].evolve(Q2)
        g =conf['pdf'].storage[Q2]['g'] 
        Nf=conf['alphaS'].get_Nf(Q2) 
        a=conf['alphaS'].get_a(Q2)  
  
        if stf=='W2':
            CQ = self.C2Q[0] + a*self.order*self.C2Q[1] 
            CG = self.C2G[0] + a*self.order*self.C2G[1]
  
        elif stf=='WL':
            CQ = a*self.order*self.CLQ[1]
            CG = a*self.order*self.CLG[1]
  
        elif stf=='W3':
            CQ = self.C3Q[0] + a*self.order*self.C3Q[1]
            CG = 0
  
        u=conf['pdf'].storage[Q2]['u']
        d=conf['pdf'].storage[Q2]['d']
        s=conf['pdf'].storage[Q2]['s']
        c=conf['pdf'].storage[Q2]['c']
        b=conf['pdf'].storage[Q2]['b']
  
        ub=conf['pdf'].storage[Q2]['ub']
        db=conf['pdf'].storage[Q2]['db']
        sb=conf['pdf'].storage[Q2]['sb']
        cb=conf['pdf'].storage[Q2]['cb']
        bb=conf['pdf'].storage[Q2]['bb']
  
        U =(CQ*u  + CG*g)
        D =(CQ*d  + CG*g) + (CQ*s  + CG*g) 
        UB=(CQ*ub + CG*g)
        DB=(CQ*db + CG*g) + (CQ*sb + CG*g)
  
        if Nf>3:
            U += CQ*c  + CG*g
            UB+= CQ*cb + CG*g
  
        #--factor of 2 follows PDG definition
        #--pdg.lbl.gov/2019/reviews/rpp2019-rev-structure-functions.pdf
        #--see equation (18.19)
        if sign==+1:
            if   stf=='W2': return 2*(D+UB)
            elif stf=='WL': return 2*(D+UB)
            elif stf=='W3': return 2*(D-UB)
        elif sign==-1:
            if   stf=='W2': return 2*(U+DB)
            elif stf=='WL': return 2*(U+DB)
            elif stf=='W3': return 2*(U-DB)


    #--nucleon  structure functions
  
    def get_FXN(self,x,Q2,stf='F2',nucleon='p'):
        if   stf=='F2': FX= x*self.get_T2CFX('F2',nucleon,Q2) 
        elif stf=='FL': FX= x*self.get_T2CFX('FL',nucleon,Q2)
        elif stf=='F3': FX=   self.get_T2CFX('F3',nucleon,Q2) 
        FX=self.mellin.invert(x,FX)
        return FX

  
    #--charge current 
    def get_WXN(self,x,Q2,stf='W2+',nucleon='p'):
        if '+' in stf:  sign=1
        if '-' in stf:  sign=-1
        if   'W2' in stf: WX= x*self.get_T2CWX('W2',nucleon,Q2,sign)
        elif 'WL' in stf: WX= x*self.get_T2CWX('WL',nucleon,Q2,sign)
        elif 'W3' in stf: WX=   self.get_T2CWX('W3',nucleon,Q2,sign)
        WX=self.mellin.invert(x,WX)
        return WX

    #--nuclear structure functions
        
    def get_FXA(self,X,Q2,stf='F2',nucleus='d'):

        if   nucleus=='d': p, n = 1,1
        elif nucleus=='h': p, n = 2,1

        FXp   = self.get_FXN(X,Q2,stf=stf,nucleon='p')
        FXn   = self.get_FXN(X,Q2,stf=stf,nucleon='n')
        return (p*FXp+n*FXn)/(p+n)



### Comments

The class `THEORY` provides a calculation of DIS structures functions at next-to-leading order in pQCD. In Collinear factorization  DIS structures functions are schematically give by 

$
{\cal F}_i (x_{\rm bj},Q^2)
      = \sum_q\int_{x_{\rm bj}}^1 
        \frac{d\xi}{\xi}
        ~e_q^2\left[C_{q,i}(\xi) f_{q/N}\left(\frac{x_{\rm bj}}{\xi}\right)
               +C_{g,i}(\xi) f_{g/N}\left(\frac{x_{\rm bj}}{\xi}\right)
         \right]
$

where ${\cal F}_{2,L,3} = F_2/x,F_L/x,F_3$. In Mellin space we can write the convolution as a product as we saw before

$
{\cal M}[{\cal F}_i] 
      = \sum_q
        ~e_q^2\left[{\cal M}[C_{q,i}] {\cal M}[f_{q/N}]
               +{\cal M}[C_{g,i}]{\cal M}[f_{g/N}] 
         \right]
$

The hard coeffs $C_{k,i}$ are available in the code upto next-to-leading order in pQCD





### Example

In [27]:
conf={}
conf['order']   = 1
conf['aux']     = AUX()
conf['mu20']     = 1.27**2
conf['mellin']  = MELLIN(npts=4)
conf['alphaS']  = ALPHAS()
conf['eweak']   = EWEAK()
conf['pdf']     = PDF()

In [28]:
thy=THEORY()

In [29]:
x=0.1
Q2=10.0
print(thy.get_FXN(x,Q2,stf='F2',nucleon='p'))
print(thy.get_FXN(x,Q2,stf='FL',nucleon='p'))
print(thy.get_FXA(x,Q2,stf='F2',nucleus='d'))
print(thy.get_WXN(x,Q2,stf='W2+',nucleon='p'))

0.3153300443099802
0.034701236098895694
0.2740780899557009
0.8073559583978069


# Fit PDFs to global DIS data


## Dataset selection

In [30]:
def get_datasets(Q2cut=1.27**2, W2cut=10, ihera=False):

    conf['datasets'] = {}
    conf['datasets']['idis'] = {}
    conf['datasets']['idis']['filters'] = []
    conf['datasets']['idis']['filters'].append("Q2>%f"%Q2cut)
    conf['datasets']['idis']['filters'].append("W2>%f"%W2cut)
    conf['datasets']['idis']['xlsx'] = {}
    #------------------------------------------------------------------------------------------------------------------
    conf['datasets']['idis']['xlsx'][10010] = 'idis/expdata/10010.xlsx' # proton   | F2            | SLAC
    conf['datasets']['idis']['xlsx'][10016] = 'idis/expdata/10016.xlsx' # proton   | F2            | BCDMS
    conf['datasets']['idis']['xlsx'][10020] = 'idis/expdata/10020.xlsx' # proton   | F2            | NMC
    if ihera:
        conf['datasets']['idis']['xlsx'][10026] = 'idis/expdata/10026.xlsx' # proton   | sigma red     | HERA II NC e+ (1)
        conf['datasets']['idis']['xlsx'][10027] = 'idis/expdata/10027.xlsx' # proton   | sigma red     | HERA II NC e+ (2)
        conf['datasets']['idis']['xlsx'][10028] = 'idis/expdata/10028.xlsx' # proton   | sigma red     | HERA II NC e+ (3)
        conf['datasets']['idis']['xlsx'][10029] = 'idis/expdata/10029.xlsx' # proton   | sigma red     | HERA II NC e+ (4)
        conf['datasets']['idis']['xlsx'][10030] = 'idis/expdata/10030.xlsx' # proton   | sigma red     | HERA II NC e-
        conf['datasets']['idis']['xlsx'][10031] = 'idis/expdata/10031.xlsx' # proton   | sigma red     | HERA II CC e+
        conf['datasets']['idis']['xlsx'][10032] = 'idis/expdata/10032.xlsx' # proton   | sigma red     | HERA II CC e-
    #-------------------------------------------------------------------------------------------------------------------
    conf['datasets']['idis']['xlsx'][10011] = 'idis/expdata/10011.xlsx' # deuteron | F2            | SLAC
    conf['datasets']['idis']['xlsx'][10017] = 'idis/expdata/10017.xlsx' # deuteron | F2            | BCDMS
    conf['datasets']['idis']['xlsx'][10021] = 'idis/expdata/10021.xlsx' # d/p      | F2d/F2p       | NMC

    #--normalization uncertainties to be fitted
    conf['datasets']['idis']['norm'] = {}
    #-------------------------------------------------------------------------------------------------------------------
    conf['datasets']['idis']['norm'][10010] ={'value':    1.03591e+00, 'min': 8.00000e-01, 'max': 1.20000e+00, 'fixed': False}
    conf['datasets']['idis']['norm'][10016] ={'value':    9.88788e-01, 'min': 8.00000e-01, 'max': 1.20000e+00, 'fixed': False}
    conf['datasets']['idis']['norm'][10020] ={'value':    1.02603e+00, 'min': 8.00000e-01, 'max': 1.20000e+00, 'fixed': False}
    #-------------------------------------------------------------------------------------------------------------------
    conf['datasets']['idis']['norm'][10011] ={'value':    1.03121e+00, 'min': 8.00000e-01, 'max': 1.20000e+00, 'fixed': False}
    conf['datasets']['idis']['norm'][10017] ={'value':    1.01588e+00, 'min': 8.00000e-01, 'max': 1.20000e+00, 'fixed': False}

### comments

The function `get_dataset` loads on `conf` the data sets we want to use to fit the pdfs. The datata sets are distinguished by arbitrary indices `idx`: `conf['datasets']['idis']['xlsx'][idx]`

We include normalization parameters to be fitted for a subset of data sets that (we will discuss later about this). Each free parameter is specified via  `{'value': ..., 'min': ..., 'max': ..., 'fixed': False}`. `min` and `max` indicates the allowed range for the parameters to vary. `fixed` indicates of the parameter is free to vary or fixed to the value specified at `value` 

`get_dataset` includes DIS kinematic cuts `Q2cut` and  `W2cut` that restrict the phase space 


## custminze loader for the experimental DIS data tables

In [31]:
class READER:
  
    def __init__(self):
        self.aux=conf['aux']
  
    def isnumeric(self,value):
        try:
            int(value)
            return True
        except:
            return False


    def get_X(self,tab):
        cols=tab.columns.values
        if any([c=='X' for c in cols])==False:
            if any([c=='W2' for c in cols]):
                tab['X']=pd.Series(tab['Q2']/(tab['W2']-self.aux.M2+tab['Q2']),index=tab.index)
            elif any([c=='W' for c in cols]):
                tab['X']=pd.Series(tab['Q2']/(tab['W']**2-self.aux.M2+tab['Q2']),index=tab.index)
            else:
                print('cannot retrive X values')
                sys.exit()
        return tab
  
    def get_Y(self,tab):
        cols=tab.columns.values
        if any([c=='Y' for c in cols])==False:
            M=self.aux.M
            if any([c=='Elab' for c in cols]):
                tab['Y']=pd.Series(tab['Q2']/(2*self.aux.M*tab['X']*tab['Elab']),index=tab.index)
            elif any([c=='E' for c in cols]):
                tab['Y']=pd.Series(tab['Q2']/(2*self.aux.M*tab['X']*tab['E']),index=tab.index)
            elif any([c=='RS' for c in cols]):
                tab['Y']=pd.Series(tab['Q2']/(tab['RS']**2-self.aux.M**2)/tab['X'],index=tab.index)
        return tab
  
    def get_W2(self,tab):
        cols=tab.columns.values
        if any([c=='W2' for c in cols])==False: 
            tab['W2'] = pd.Series(self.aux.M2 + tab.Q2/tab.X - tab.Q2,index=tab.index)
        return tab
  
    def get_idx(self,tab):
        tab['idx']=pd.Series(tab.index,index=tab.index)
        return tab
  
    def modify_table(self,tab):
        tab=self.get_X(tab)   
        tab=self.get_Y(tab)   
        tab=self.get_W2(tab)   
        tab=self.apply_cuts(tab)
        tab=self.get_idx(tab)
        return tab
    
    def apply_cuts(self,tab):
        if  'filters' in conf['datasets'][self.reaction]:
            for f in conf['datasets'][self.reaction]['filters']:
                tab=tab.query(f)
        return tab

    def load_data_sets(self,reaction,verb=True):
        self.reaction=reaction
        if reaction not in conf['datasets']: return None
        XLSX=conf['datasets'][reaction]['xlsx']
        TAB={}
        for k in XLSX: 
            if verb: print('loading %s data sets %d'%(reaction,k))
            fname=conf['datasets'][reaction]['xlsx'][k]
            if  fname.startswith('./'):
                tab=pd.read_excel(fname)
            else:
                tab=pd.read_excel('./database/%s'%(fname))
            tab=self.modify_table(tab)
            npts=tab.index.size
            if npts==0: continue
            TAB[k]=tab.to_dict(orient='list')
            for kk in TAB[k]: 
                if  self.isnumeric(TAB[k][kk][0]):
                    if kk == 'idx':
                        TAB[k][kk]=np.array(TAB[k][kk])
                    else:
                        TAB[k][kk]=np.array(TAB[k][kk]).astype(np.float64)
     
        return TAB    
    

### Comments

The class `READER` loads the datasets inside `conf['datasets']['idis']`. Some datasets does not include the full set of kinematic variables $x,y,W^2$, and the loader adds missing variables as columns. In addition using the kinematic cuts specified in `get_dataset`, the loader will remove the data points that are outside of the cuts. 


### Example

In [32]:
conf={}
conf['aux']=AUX()
get_datasets(Q2cut=1.27**2, W2cut=10, ihera=True)
tabs=READER().load_data_sets('idis')

loading idis data sets 10010
loading idis data sets 10016
loading idis data sets 10020
loading idis data sets 10026
loading idis data sets 10027
loading idis data sets 10028
loading idis data sets 10029
loading idis data sets 10030
loading idis data sets 10031
loading idis data sets 10032
loading idis data sets 10011
loading idis data sets 10017
loading idis data sets 10021


In [33]:
tabs.keys()

dict_keys([10010, 10016, 10020, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10011, 10017, 10021])

In [34]:
pd.DataFrame(tabs[10010])

,col,X,Q2,obs,target,lepton beam,current,units,value,stat_u,syst_u,*norm_c,W2,idx
0,SLAC,0.075,1.66,F2,p,e,NC,1.0,0.35448,0.01028,0.00461,0.007444,21.354902,6
1,SLAC,0.077,2.01,F2,p,e,NC,1.0,0.37937,0.01328,0.00910,0.007967,24.975465,10
2,SLAC,0.080,1.67,F2,p,e,NC,1.0,0.35137,0.01476,0.00984,0.007379,20.086569,12
3,SLAC,0.083,1.76,F2,p,e,NC,1.0,0.34599,0.00588,0.00934,0.007266,20.326388,17
4,SLAC,0.083,2.39,F2,p,e,NC,1.0,0.35900,0.01185,0.00933,0.007539,27.286750,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,SLAC,0.633,17.69,F2,p,e,NC,NaN,0.05403,0.00259,0.00362,0.001135,11.137856,510
218,SLAC,0.658,22.12,F2,p,e,NC,NaN,0.04110,0.00148,0.00218,0.000863,12.378590,528
219,SLAC,0.667,22.93,F2,p,e,NC,NaN,0.03799,0.00125,0.00251,0.000798,12.329380,533
220,SLAC,0.708,23.66,F2,p,e,NC,NaN,0.02551,0.00184,0.00138,0.000536,10.639648,570


In [35]:
data={_:[] for _ in ['idx','col','target','current','obs']}

for idx in tabs:
    tab=tabs[idx]
    data['idx'].append(idx)
    data['col'].append(tab['col'][0])
    data['target'].append(tab['target'][0])
    data['current'].append(tab['current'][0])
    data['obs'].append(tab['obs'][0])
pd.DataFrame(data)

,idx,col,target,current,obs
0,10010,SLAC,p,NC,F2
1,10016,BCDMS,p,NC,F2
2,10020,NMC,p,NC,F2
3,10026,HERA II NC e+ (1),p,NC,sig_r
4,10027,HERA II NC e+ (2),p,NC,sig_r
5,10028,HERA II NC e+ (3),p,NC,sig_r
6,10029,HERA II NC e+ (4),p,NC,sig_r
7,10030,HERA II NC e-,p,NC,sig_r
8,10031,HERA II CC e+,p,CC,sig_r
9,10032,HERA II CC e-,p,CC,sig_r


### Exercise 7 (time: 5 mins)

- plot the kinematics bins $x_{\rm bj},~Q^2$ of the world DIS data sets
- **Hint**: use log scale for both axis 

## The loss function

In [36]:
class LOSS:

    def __init__(self): 
        
        self.reaction = 'idis'
        self.tabs     = conf['idis tabs']
        if 'idis' in conf:
            self.thy      = conf['idis']
        self.setup()
 
    def percent_to_absolute(self):

        for k in self.tabs:
            ucorr = [x for x in self.tabs[k] if '_u' in x and '%' in x]
            corr  = [x for x in self.tabs[k] if '_c' in x and '%' in x]

            if  len(ucorr)!=0:
                for name in ucorr:
                    mod_name=name.replace('%','')
                    self.tabs[k][mod_name]=self.tabs[k]['value'] * self.tabs[k][name]/100.0

            if  len(corr)!=0:
                for name in corr:
                    mod_name=name.replace('%','')
                    self.tabs[k][mod_name]=self.tabs[k]['value'] * self.tabs[k][name]/100.0
  
    def add_columns(self):

        for k in self.tabs:
            npts=len(self.tabs[k]['value'])
            self.tabs[k]['thy']=np.zeros(npts)
            self.tabs[k]['N']=np.zeros(npts)
            self.tabs[k]['residuals']=np.zeros(npts)
            self.tabs[k]['r-residuals']=np.zeros(npts)
            self.tabs[k]['Shift']=np.zeros(npts)
  
    def get_alpha(self):

        for k in self.tabs:
            npts=len(self.tabs[k]['value'])
            alpha2=np.zeros(npts) 
            ucorr = [x for x in self.tabs[k] if '_u' in x and '%' not in x]
            for kk in ucorr: alpha2+=self.tabs[k][kk]**2
            self.tabs[k]['alpha']=alpha2**0.5
  
    def retrieve_norm_uncertainty(self):
  
        for k in self.tabs:

            if k not in conf['datasets'][self.reaction]['norm']: continue

            norm  = [_ for _ in self.tabs[k] if '_c' in _ and 'norm' in _ and '%' not in _]

            if  len(norm)>1:
                msg='ERR: more than one normalization found at %s %d'%(self.reaction,k)
                raise ValueError(msg)

            elif len(norm)==1:
                print('%d has norm uncertainty'%k)
                for i in range(len(self.tabs[k]['value'])):
                    if self.tabs[k]['value'][0]!=0:
                        dN=self.tabs[k][norm[0]][i]/(self.tabs[k]['value'][i])
                        break
                conf['datasets'][self.reaction]['norm'][k]['dN']=dN

            elif len(norm)==0:
                print('Cannot read normalization uncertainty for data set ',self.reaction,k)
                sys.exit()
  
    def setup_rparams(self):

        if  'rparams' not in conf: 
            conf['rparams']={}

        if  self.reaction not in conf['rparams']: 
            conf['rparams'][self.reaction]={}

        for k in self.tabs:

            if  k not in conf['rparams'][self.reaction]:
                conf['rparams'][self.reaction][k]={}

            corr = [x for x in self.tabs[k] if '_c' in x and '%' not in x and 'norm' not in x]
            for c in corr: 
                conf['rparams'][self.reaction][k][c]={'value':0.0,'registered':False}
  
    def resample(self):
        self.tabs=copy.deepcopy(self.original)
        for k in self.tabs:
            npts=len(self.tabs[k]['value'])
            self.tabs[k]['value']+=randn(npts)*self.tabs[k]['alpha']
  
    #--master setup
  
    def setup(self):
        self.percent_to_absolute()
        self.add_columns()
        self.get_alpha()
        self.retrieve_norm_uncertainty()
        self.setup_rparams()
        self.original=copy.deepcopy(self.tabs)
        if 'bootstrap' in conf and conf['bootstrap']: self.resample()
  
    #--residuals
  
    def _get_residuals(self,k):

        npts=len(self.tabs[k]['value'])
        exp=self.tabs[k]['value']

        if  k in conf['datasets'][self.reaction]['norm']:
            norm=conf['datasets'][self.reaction]['norm'][k]['value']
        else:
            norm=1.0

        thy=self.tabs[k]['thy']/norm
        alpha=self.tabs[k]['alpha']
        corr = [x for x in self.tabs[k] if '_c' in x and '%' not in x and 'norm' not in x]
        N=np.ones(exp.size)
        ncorr=len(corr)

        if  ncorr==0:

            self.tabs[k]['N']=N
            self.tabs[k]['residuals']=(exp-thy)/alpha
            self.tabs[k]['shift']=np.zeros(exp.size)
            self.tabs[k]['prediction']=thy

        else:

            beta=[self.tabs[k][c] * (thy/(exp+1e-100)) for c in corr]
            A=np.diag(np.diag(np.ones((ncorr,ncorr)))) + np.einsum('ki,li,i->kl',beta,beta,1/alpha**2)
            B=np.einsum('ki,i,i->k',beta,exp-thy,1/alpha**2)
            try:
                r=np.einsum('kl,l->k',np.linalg.inv(A),B)
            except:
                r=np.zeros(len(beta))
            shift=np.einsum('k,ki->i',r,beta)
            for i in range(ncorr):
                conf['rparams'][self.reaction][k][corr[i]]['value']=r[i]
            self.tabs[k]['N']=N
            self.tabs[k]['residuals']=(exp-shift-thy)/alpha
            self.tabs[k]['shift']=shift
            self.tabs[k]['prediction']=shift+thy

        return self.tabs[k]['residuals']
  
    def _get_rres(self,k):
        rres=[]
        rparams=conf['rparams'][self.reaction][k]
        for c in rparams:
            rres.append(rparams[c]['value'])
        return np.array(rres)
  
    def _get_nres(self,k):

        if  k not in conf['datasets'][self.reaction]['norm']:
            return 0

        elif  conf['datasets'][self.reaction]['norm'][k]['fixed']:
            return 0

        else:
            norm=conf['datasets'][self.reaction]['norm'][k]
            return (norm['value']-1)/norm['dN']
  
    def get_residuals(self):

        res,rres,nres=[],[],[]

        self.get_theory()

        for k in self.tabs: 
            res=np.append(res,self._get_residuals(k))
            rres=np.append(rres,self._get_rres(k))
            nres=np.append(nres,self._get_nres(k))
        return res,rres,nres
  
    #--mis functions 
 
    def get_npts(self):
        npts=0
        for k in self.tabs: 
            npts+=len(self.tabs[k]['value'])
        return npts

    def get_chi2(self):
        data={self.reaction:{}}
        for idx in self.tabs:
            if len(self.tabs[idx])==0: continue 
            res=self.tabs[idx]['residuals']
            npts=res.size
            chi2=np.sum(res**2)
            data[self.reaction][idx]={'npts':npts,'chi2':chi2}
        return data

### Comments

The class LOSS builds the $\chi^2$ function with the following functional form  

$
\chi^2(p) = \sum_{e,i} 
          \left(\frac{d_{e,i}-\sum_k r_{e,k} \beta_{k,i}-T_{e,i}(p)/N_e}{\alpha_i}
          \right)^2
         + \sum_k r_{e,k}^2 + \left(\frac{1-N_e}{\delta N_e}\right)^2
$

- $e$: label for a data set
- $i$: label for a data point 
- $d_{e,i}$: central value of the observable in the column `value`
- $T_{e,i}$: corresponding theoretical value. When computed it will be stored in the column `thy` 
- $\beta_{k,i}$: k-th source of point-by-point correlated systematic uncertainty. The are the columns that has `_c` except if the column name includes the word `norm`.
- $\alpha_i$: sum in quadrate of all uncorrelated uncertainties from columns ending with `_u`
- $\delta N_e$: normalization uncertainty from the column with word `norm`.
- $r_{e,k}$: k nuisance free parameters to be tunned  
- $N_e$: normalization free parameter to be tunned

Since $r_{e,k}$ are pure quardatic terms in the $\chi^2$ function, the can be optimized analytically using $\partial \chi^2/\partial r_{e,k}=0$. See appendix B in https://inspirehep.net/literature/581996 for more details. 

The quantity $\sum_k r_{e,k} \beta_{k,i}+T_{e,i}/N_e$ in the $\chi^2$ function should be viewed as the effective theory that includes additive and multiplicative distortions of the pure theoretical quantity  $T_{e,i}$. These distortions are controlled by the nuisance parameters $r_{e,k}$ and $N_e$ and they are distributed Gaussianly as indicated by the penalty terms in the $\chi^2$ function. 

### Example

In [37]:
conf={}
conf['aux']=AUX()
get_datasets(Q2cut=1.27**2, W2cut=10, ihera=False)
conf['idis tabs']=READER().load_data_sets('idis')
loss=LOSS()

loading idis data sets 10010
loading idis data sets 10016
loading idis data sets 10020
loading idis data sets 10011
loading idis data sets 10017
loading idis data sets 10021
10010 has norm uncertainty
10016 has norm uncertainty
10020 has norm uncertainty
10011 has norm uncertainty
10017 has norm uncertainty


In [38]:
loss.tabs.keys()

dict_keys([10010, 10016, 10020, 10011, 10017, 10021])

In [39]:
pd.DataFrame(loss.tabs[10010])

,col,X,Q2,obs,target,lepton beam,current,units,value,stat_u,syst_u,*norm_c,W2,idx,thy,N,residuals,r-residuals,Shift,alpha
0,SLAC,0.075,1.66,F2,p,e,NC,1.0,0.35448,0.01028,0.00461,0.007444,21.354902,6,0.0,0.0,0.0,0.0,0.0,0.011266
1,SLAC,0.077,2.01,F2,p,e,NC,1.0,0.37937,0.01328,0.00910,0.007967,24.975465,10,0.0,0.0,0.0,0.0,0.0,0.016099
2,SLAC,0.080,1.67,F2,p,e,NC,1.0,0.35137,0.01476,0.00984,0.007379,20.086569,12,0.0,0.0,0.0,0.0,0.0,0.017739
3,SLAC,0.083,1.76,F2,p,e,NC,1.0,0.34599,0.00588,0.00934,0.007266,20.326388,17,0.0,0.0,0.0,0.0,0.0,0.011037
4,SLAC,0.083,2.39,F2,p,e,NC,1.0,0.35900,0.01185,0.00933,0.007539,27.286750,18,0.0,0.0,0.0,0.0,0.0,0.015082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,SLAC,0.633,17.69,F2,p,e,NC,NaN,0.05403,0.00259,0.00362,0.001135,11.137856,510,0.0,0.0,0.0,0.0,0.0,0.004451
218,SLAC,0.658,22.12,F2,p,e,NC,NaN,0.04110,0.00148,0.00218,0.000863,12.378590,528,0.0,0.0,0.0,0.0,0.0,0.002635
219,SLAC,0.667,22.93,F2,p,e,NC,NaN,0.03799,0.00125,0.00251,0.000798,12.329380,533,0.0,0.0,0.0,0.0,0.0,0.002804
220,SLAC,0.708,23.66,F2,p,e,NC,NaN,0.02551,0.00184,0.00138,0.000536,10.639648,570,0.0,0.0,0.0,0.0,0.0,0.002300


### Exercise 8 (time: 5 mins)

- plot the values of $F^{{\rm NC}}_{2,p}$ as a function of $x_{\rm bj}$ from all the data sets 
- **Hint**: use the colums `X`, `value`, `alpha` from `loss.tabs`

## DIS tailored residuals calculator 

In [40]:
class RESIDUALS(LOSS):

    def _get_sigma_r(self,idx,F2,FL,F3,i):
  
        if   '_minus' in self.tabs[idx]['lepton beam'][0]: sign = 1
        elif '_plus'  in self.tabs[idx]['lepton beam'][0]: sign = -1
        elif 'none'   in self.tabs[idx]['lepton beam'][0]: sign = 0
  
        else: 
            print('ERR in data set %d: lepton id is not identified'%(idx))
            sys.exit()
  
        x  = self.tabs[idx]['X'][i]
        Q2 = self.tabs[idx]['Q2'][i]
        M2 = conf['aux'].M2
  
        if 'Elab' in self.tabs[idx]:
            ELab=self.tabs[idx]['Elab'][i]
            s=M2+2*ELab*M2**0.5
            y=(Q2/2/x)/((s-M2)/2)
        elif 'Y' in self.tabs[idx]:
            y=self.tabs[idx]['Y'][i]

        YP=1+(1-y)**2
        YM=1-(1-y)**2

        thy=F2-y**2/YP*FL+sign*YM/YP*x*F3
        return thy
  
    def _get_sigma_rcc(self,idx,W2,WL,W3,i):
  
        if    '_plus'  in self.tabs[idx]['lepton beam'][i]: sign=-1
        elif  '_minus' in self.tabs[idx]['lepton beam'][i]: sign=1
        else: 
            print('ERR in data set %d: lepton id is not identified'%(idx))
            sys.exit()
  
        x=self.tabs[idx]['X'][i]
        Q2=self.tabs[idx]['Q2'][i]
        M2=conf['aux'].M2
  
        if 'Elab' in self.tabs[idx]:
            ELab=self.tabs[idx]['Elab'][i]
            s=M2+2*ELab*M2**0.5
            y=(Q2/2/x)/((s-M2)/2)
        elif 'Y' in self.tabs[idx]:
            y=self.tabs[idx]['Y'][i]

        YP=1+(1-y)**2
        YM=1-(1-y)**2
  
        thy=YP/4*W2-y**2/4*WL+sign*YM/4*x*W3
        return thy
  
    def get_sigma_r(self,x,Q2,idx,tar,i):

        current=self.tabs[idx]['current'][0].strip()

        if current=='NC':

            F2  = self.thy.get_FXN(x,Q2,stf='F2',nucleon=tar)
            FL  = self.thy.get_FXN(x,Q2,stf='FL',nucleon=tar)
            F3  = self.thy.get_FXN(x,Q2,stf='F3',nucleon=tar)
            thy = self._get_sigma_r(idx,F2,FL,F3,i)
  
        if current=='CC':
  
            if   '_plus'  in self.tabs[idx]['lepton beam'][0]:
                sign = 1
                W2   = self.thy.get_WXN(x,Q2,stf='W2+',nucleon=tar)
                WL   = self.thy.get_WXN(x,Q2,stf='WL+',nucleon=tar)
                W3   = self.thy.get_WXN(x,Q2,stf='W3+',nucleon=tar)

            elif '_minus' in self.tabs[idx]['lepton beam'][0]:
                sign = -1
                W2   = self.thy.get_WXN(x,Q2,stf='W2-',nucleon=tar)
                WL   = self.thy.get_WXN(x,Q2,stf='WL-',nucleon=tar)
                W3   = self.thy.get_WXN(x,Q2,stf='W3-',nucleon=tar)
    
            thy = self._get_sigma_rcc(idx,W2,WL,W3,i)

        return thy

    def get_F2d_over_F2p(self,x,Q2):
        F2p=self.thy.get_FXN(x,Q2,stf='F2',nucleon='p')
        F2d=self.thy.get_FXA(x,Q2,stf='F2',nucleus='d')
        return F2d/F2p

    def get_F2n_over_F2d(self,x,Q2):
        F2n=self.thy.get_FXN(x,Q2,stf='F2',nucleon='n')
        F2d=2*self.thy.get_FXA(x,Q2,stf='F2',nucleus='d')
        return F2n/F2d
 
    def get_theory(self):

        for idx in self.tabs:

            for i in range(self.tabs[idx]['value'].size):
                x=self.tabs[idx]['X'][i]
                Q2=self.tabs[idx]['Q2'][i]
                target=self.tabs[idx]['target'][i]
                obs=self.tabs[idx]['obs'][i].strip()

                if   obs=='F2':
                    if target=='p':  thy=self.thy.get_FXN(x,Q2,stf='F2',nucleon='p')
                    if target=='d':  thy=self.thy.get_FXA(x,Q2,stf='F2',nucleus='d')
                elif obs=='F2d/F2p': thy=self.get_F2d_over_F2p(x,Q2)
                elif obs=='F2n/F2d': thy=self.get_F2n_over_F2d(x,Q2)
                elif obs=='sig_r' :  thy=self.get_sigma_r(x,Q2,idx,target,i)
                else:
                    msg='exp=%d obs=%s and target=%s not implemented'
                    raise ValueError(msg%(idx,obs,target))  
                self.tabs[idx]['thy'][i]=thy
  
    def gen_report(self):

        msg ='reaction: unpol DIS <br />'
        for f in conf['datasets']['idis']['filters']:
            msg+='filters: %s <br />'%f
        msg+='reaction: unpol DIS <br />'
 

        
        data={_:[] for _ in ['idx','col','obs','tar','npts','chi2','chi2/npts','rchi2','nchi2']}
        for k in self.tabs:
            if len(self.tabs[k])==0: continue 
            res=self.tabs[k]['residuals']

            rres=[]
            for c in conf['rparams']['idis'][k]:
                rres.append(conf['rparams']['idis'][k][c]['value'])
            rres=np.array(rres)

            if k in conf['datasets']['idis']['norm']:
                norm=conf['datasets']['idis']['norm'][k]
                nres=(norm['value']-1)/norm['dN']
            else:
                nres=0

            chi2=np.sum(res**2)
            rchi2=np.sum(rres**2)
            nchi2=nres**2
            tar=self.tabs[k]['target'][0]
            col=self.tabs[k]['col'][0].split()[0]
            obs=self.tabs[k]['obs'][0].strip()
            npts=res.size
            data['idx'].append('%7d'%k)
            data['col'].append('%10s'%col)
            data['obs'].append('%3s'%obs)
            data['tar'].append('%3s'%tar)
            data['npts'].append('%10.2f'%npts)
            data['chi2'].append('%10.2f'%chi2)
            data['chi2/npts'].append('%10.2f'%(chi2/npts))
            data['rchi2'].append('%10.2f'%rchi2)
            data['nchi2'].append('%10.2f'%nchi2)
            
        data=pd.DataFrame(data)
        msg+=data.to_html(col_space=80,index=False)#,justify='center')
        return msg


### Comments

The `RESIDUALS` class built upon the `LOSS` class is tailored to DIS data that includes methods to evaluate specific observables using the `THEORY` class. Its main role is to update the column `thy` inside the tables in order to compute the residuals

### Example

In [41]:
conf={}
get_datasets(Q2cut=1.27**2, W2cut=10, ihera=False)
conf['order']     = 1
conf['aux']       = AUX()
conf['mu20']       = 1.27**2
conf['mellin']    = MELLIN(npts=4)
conf['alphaS']    = ALPHAS()
conf['eweak']     = EWEAK()
conf['pdf']       = PDF()
conf['idis tabs'] = READER().load_data_sets('idis')
conf['idis']      = THEORY()
residuals=RESIDUALS()

loading idis data sets 10010
loading idis data sets 10016
loading idis data sets 10020
loading idis data sets 10011
loading idis data sets 10017
loading idis data sets 10021
10010 has norm uncertainty
10016 has norm uncertainty
10020 has norm uncertainty
10011 has norm uncertainty
10017 has norm uncertainty


In [42]:
res,rres,nres =residuals.get_residuals()
print('res :',res.shape)
print('rres:',rres.shape)
print('nres:',nres.shape)

res : (1503,)
rres: (26,)
nres: (6,)


The method `get_residuals` inherited from the `LOSS` class returns 3 arrays 

`res`$=\frac{d_{e,i}-\sum_k r_{e,k} \beta_{k,i}-T_{e,i}(p)/N_e}{\alpha_i}$ for all $e,i$

`rres`$=r_{e,k}$ for all $e,k$

`nres`$=\frac{1-N_e}{\delta N_e}$  for all $e,k$

The shapes printed above indicates that there are 1503 data experimental DIS data points, 26 nuisance parameters and 6 normalization parameters  


In [43]:
widgets.HTML(
    value=residuals.gen_report(),
    placeholder='Some HTML',
    description='',
)

HTML(value='reaction: unpol DIS <br />filters: Q2>1.612900 <br />filters: W2>10.000000 <br />reaction: unpol D…

### Exercise 9 (time: 5 mins)

- Include the HERA datasets via `get_datasets(Q2cut=1.27**2, W2cut=10, ihera=True)`
- Print the lenght of `res`,`rres`,`nres` and interpret these numbers
- Print the summary via `residuals.gen_report`

## Parameters Manager (PARMAN)

So far we have use default parameters values of the pdf class to carry out theory calculations. Our goal however is to be able to tune the pdf parameters to describe the data as best as possible. In what follows we explain our code design to that allows to manipulate the pdf parameters.

We start by defining new entries on `conf` associated with the parameters we want to fit 

In [44]:
def setup_params():

    conf['params'] = {}

    conf['params']['pdf'] = {}

    conf['params']['pdf']['g N']    ={'value':    3.09994e-01, 'min':  None, 'max':  None, 'fixed': True }
    conf['params']['pdf']['g a']    ={'value':   -5.20900e-01, 'min':  -1.9, 'max':     1, 'fixed': False}
    conf['params']['pdf']['g b']    ={'value':    4.29360e+00, 'min':     0, 'max':    10, 'fixed': False}

    conf['params']['pdf']['uv N']   ={'value':    3.25322e-01, 'min':  None, 'max':  None, 'fixed': True }
    conf['params']['pdf']['uv a']   ={'value':   -2.14402e-01, 'min':  -0.6, 'max':     1, 'fixed': False}
    conf['params']['pdf']['uv b']   ={'value':    3.04406e+00, 'min':     0, 'max':    10, 'fixed': False}

    conf['params']['pdf']['dv N']   ={'value':    1.06672e-01, 'min':  None, 'max':  None, 'fixed': True }
    conf['params']['pdf']['dv a']   ={'value':   -3.45404e-01, 'min':  -0.6, 'max':     1, 'fixed': False}
    conf['params']['pdf']['dv b']   ={'value':    4.48193e+00, 'min':     0, 'max':    10, 'fixed': False}

    conf['params']['pdf']['db N']   ={'value':    3.65346e-02, 'min':     0, 'max':     1, 'fixed': False}
    conf['params']['pdf']['db a']   ={'value':   -9.35028e-01, 'min':    -1, 'max':     1, 'fixed': False}
    conf['params']['pdf']['db b']   ={'value':    4.48545e+00, 'min':     0, 'max':    10, 'fixed': False}

    conf['params']['pdf']['ub N']   ={'value':    1.70043e-02, 'min':     0, 'max':     1, 'fixed': False}
    conf['params']['pdf']['ub a']   ={'value':   -1.00000e+00, 'min':    -1, 'max':     1, 'fixed': False}
    conf['params']['pdf']['ub b']   ={'value':    1.00000e+01, 'min':     0, 'max':    10, 'fixed': False}

    conf['params']['pdf']['s N']    ={'value':    9.91077e-02, 'min':     0, 'max':     1, 'fixed': True}
    conf['params']['pdf']['s a']    ={'value':    1.00000e+00, 'min':  -0.6, 'max':     1, 'fixed': False}
    conf['params']['pdf']['s b']    ={'value':    4.43290e+00, 'min':     0, 'max':    10, 'fixed': False}

    conf['params']['pdf']['sb N']   ={'value':    2.96987e-02, 'min':     0, 'max':     1, 'fixed': False}
    conf['params']['pdf']['sb a']   ={'value':   -6.00000e-01, 'min':  -0.6, 'max':     1, 'fixed': False}
    conf['params']['pdf']['sb b']   ={'value':    3.56087e+00, 'min':     0, 'max':    10, 'fixed': False}

    conf['params']['pdf']['sea1 N']  ={'value':    3.68792e-03, 'min':     0, 'max':     1, 'fixed': False}
    conf['params']['pdf']['sea1 a']  ={'value':   -1.87906e+00, 'min':  -1.9, 'max':    -1, 'fixed': False}
    conf['params']['pdf']['sea1 b']  ={'value':    8.07746e+00, 'min':     0, 'max':    10, 'fixed': False}

    conf['params']['pdf']['sea2 N']  ={'value':    3.68792e-03, 'min':     0, 'max':     1, 'fixed': 'sea1 N'}
    conf['params']['pdf']['sea2 a']  ={'value':   -1.87906e+00, 'min':  -1.9, 'max':    -1, 'fixed': 'sea1 a'}
    conf['params']['pdf']['sea2 b']  ={'value':    8.07746e+00, 'min':     0, 'max':    10, 'fixed': 'sea1 b'}


Here we list all the parameters we are interested and set the allowed range via  `min` and `max`. We also include the key `fixed` which means 

- `fixed=True`: This parameter is fixed to the current numerical entry at `value`
- `fixed=True`: This means the parameter is free
- `fixed=str`:  One case chose to equate the parameter to another parameter e.g 

An example for `fixed=str` is 

`conf['params']['pdf']['sb b'] ={'value': 3.56087e+00, 'min':0, 'max':10, 'fixed':'s b'}`

which means we set the exponent $(1-x)^b$ of the template shape for `sb` to be the same as the one of `s`.

We proceed next to build a class that updates the parameters of the class `PDF`


In [45]:
class PARMAN:

    def __init__(self):
        self.get_ordered_free_params()

    def get_ordered_free_params(self):
        self.par=[]
        self.order=[]
        self.pmin=[]
        self.pmax=[]

        if 'check lims' not in conf: conf['check lims']=True

        for k in conf['params']:
            for kk in conf['params'][k]:
                if  conf['params'][k][kk]['fixed']==False:
                    p=conf['params'][k][kk]['value']
                    pmin=conf['params'][k][kk]['min']
                    pmax=conf['params'][k][kk]['max']
                    self.pmin.append(pmin)
                    self.pmax.append(pmax)
                    if p<pmin or p>pmax:
                        if conf['check lims']: raise ValueError('par limits are not consistend with central: %s %s'%(k,kk))

                    self.par.append(p)
                    self.order.append([1,k,kk])

        if 'datasets' in conf:
            for k in conf['datasets']:
                for kk in conf['datasets'][k]['norm']:
                    if  conf['datasets'][k]['norm'][kk]['fixed']==False:
                        p=conf['datasets'][k]['norm'][kk]['value']
                        pmin=conf['datasets'][k]['norm'][kk]['min']
                        pmax=conf['datasets'][k]['norm'][kk]['max']
                        self.pmin.append(pmin)
                        self.pmax.append(pmax)
                        if p<pmin or p>pmax:
                            if conf['check lims']: raise ValueError('par limits are not consistend with central: %s %s'%(k,kk))
                        self.par.append(p)
                        self.order.append([2,k,kk])

        self.pmin=np.array(self.pmin)
        self.pmax=np.array(self.pmax)
        self.par=np.array(self.par)
        self.set_new_params(self.par,initial=True)

    def gen_flat(self,setup=True):
        r=np.random.uniform(0,1,len(self.par))
        par=self.pmin + r * (self.pmax-self.pmin)
        if setup: self.set_new_params(par,initial=True)
        return par

    def check_lims(self):
        flag=True
        for k in conf['params']:
            for kk in conf['params'][k]:
                if  conf['params'][k][kk]['fixed']==False:
                    p=conf['params'][k][kk]['value']
                    pmin=conf['params'][k][kk]['min']
                    pmax=conf['params'][k][kk]['max']
                    if  p<pmin or p>pmax:
                        print(k,kk, p,pmin,pmax)
                        flag=False

        if  'datasets' in conf:
            for k in conf['datasets']:
                for kk in conf['datasets'][k]['norm']:
                    if  conf['datasets'][k]['norm'][kk]['fixed']==False:
                        p=conf['datasets'][k]['norm'][kk]['value']
                        pmin=conf['datasets'][k]['norm'][kk]['min']
                        pmax=conf['datasets'][k]['norm'][kk]['max']
                        if p<pmin or p>pmax:
                            flag=False
                            print(k,kk, p,pmin,pmax)

        return flag

    def set_new_params(self,parnew,initial=False):
        self.par=parnew
        self.shifts=0
        semaphore={}

        for i in range(len(self.order)):
            ii,k,kk=self.order[i]
            if  ii==1:
                if k not in semaphore: semaphore[k]=0
                if  conf['params'][k][kk]['value']!=parnew[i]:
                    conf['params'][k][kk]['value']=parnew[i]
                    semaphore[k]=1
                    self.shifts+=1
            elif ii==2:
                if kk in conf['datasets'][k]['norm']:
                    if  conf['datasets'][k]['norm'][kk]['value']!=parnew[i]:
                        conf['datasets'][k]['norm'][kk]['value']=parnew[i]
                        self.shifts+=1

        if  initial:
            for k in conf['params']: semaphore[k]=1

        self.propagate_params(semaphore)

    def gen_report(self):
        data={_:[] for _ in ['idx','dist','type','value']}
        cnt=0
        for k in conf['params']:
            for kk in sorted(conf['params'][k]):
                if  conf['params'][k][kk]['fixed']==False:
                    cnt+=1
                    data['idx'].append('%d'%cnt)
                    data['dist'].append('%10s'%k)
                    data['type'].append('%10s'%kk)
                    data['value'].append('%10.2e'%conf['params'][k][kk]['value'])
                    
        data=pd.DataFrame(data)
        msg=data.to_html(col_space=80,index=False,justify='left')
        return msg

    def propagate_params(self,semaphore):
        flag=False
        if 'pdf' in semaphore and semaphore['pdf']==1: self.set_pdf_params()

    def set_params(self,dist,FLAV,PAR):

        #--setup the constraints
        for flav in FLAV:
            for par in PAR:
                if flav+' '+par not in conf['params'][dist]: continue
                if conf['params'][dist][flav+' '+par]['fixed']==True: continue
                if conf['params'][dist][flav+' '+par]['fixed']==False: continue
                reference_flav=conf['params'][dist][flav+' '+par]['fixed']

                if len(reference_flav.split())==2:
                    conf['params'][dist][flav+' '+par]['value']=conf['params'][dist][reference_flav]['value']

                elif len(reference_flav.split())==3:  #allows one to reference from another distribution
                    reference_dist=reference_flav.split()[0]
                    reference_flav=reference_flav.split()[1] + ' ' + reference_flav.split()[2] 
                    conf['params'][dist][flav+' '+par]['value']=conf['params'][reference_dist][reference_flav]['value']

        #--update values at the class
        for flav in FLAV:
            idx=0
            for par in PAR:
                if  flav+' '+par in conf['params'][dist]:
                    conf[dist].params[flav][idx]=conf['params'][dist][flav+' '+par]['value']
                else:
                    conf[dist].params[flav][idx]=0
                idx+=1
        conf[dist].setup()

        #--update values at conf
        for flav in FLAV:
            idx=0
            for par in PAR:
                if  flav+' '+par in conf['params'][dist]:
                    conf['params'][dist][flav+' '+par]['value']= conf[dist].params[flav][idx]
                idx+=1 

    def set_pdf_params(self):
        FLAV=conf['pdf'].FLAV
        PAR=conf['pdf'].PAR
        self.set_params('pdf',FLAV,PAR)


### Example

In [46]:
conf={}
setup_params()
conf['order']     = 1
conf['aux']       = AUX()
conf['mu20']      = 1.27**2
conf['mellin']    = MELLIN(npts=4)
conf['alphaS']    = ALPHAS()
conf['pdf']       = PDF()
parman=PARMAN()

Lets see the current **vetor** of parameters

In [47]:
len(parman.par)

20

In [48]:
parman.par

array([-5.20900e-01,  4.29360e+00, -2.14402e-01,  3.04406e+00,
       -3.45404e-01,  4.48193e+00,  3.65346e-02, -9.35028e-01,
        4.48545e+00,  1.70043e-02, -1.00000e+00,  1.00000e+01,
        1.00000e+00,  4.43290e+00,  2.96987e-02, -6.00000e-01,
        3.56087e+00,  3.68792e-03, -1.87906e+00,  8.07746e+00])

In [49]:
parman.order

[[1, 'pdf', 'g a'],
 [1, 'pdf', 'g b'],
 [1, 'pdf', 'uv a'],
 [1, 'pdf', 'uv b'],
 [1, 'pdf', 'dv a'],
 [1, 'pdf', 'dv b'],
 [1, 'pdf', 'db N'],
 [1, 'pdf', 'db a'],
 [1, 'pdf', 'db b'],
 [1, 'pdf', 'ub N'],
 [1, 'pdf', 'ub a'],
 [1, 'pdf', 'ub b'],
 [1, 'pdf', 's a'],
 [1, 'pdf', 's b'],
 [1, 'pdf', 'sb N'],
 [1, 'pdf', 'sb a'],
 [1, 'pdf', 'sb b'],
 [1, 'pdf', 'sea1 N'],
 [1, 'pdf', 'sea1 a'],
 [1, 'pdf', 'sea1 b']]

the array `parman.order` list the ordering of the parameters. Notice that the parameters are only those associated with the pdf parameters. Lets next load also the datasets and see the parameter list

In [50]:
conf={}
setup_params()
get_datasets(Q2cut=1.27**2, W2cut=10, ihera=True)
conf['order']     = 1
conf['aux']       = AUX()
conf['mu20']      = 1.27**2
conf['mellin']    = MELLIN(npts=4)
conf['alphaS']    = ALPHAS()
conf['pdf']       = PDF()
parman=PARMAN()

In [51]:
len(parman.par)

25

In [52]:
parman.par

array([-5.20900e-01,  4.29360e+00, -2.14402e-01,  3.04406e+00,
       -3.45404e-01,  4.48193e+00,  3.65346e-02, -9.35028e-01,
        4.48545e+00,  1.70043e-02, -1.00000e+00,  1.00000e+01,
        1.00000e+00,  4.43290e+00,  2.96987e-02, -6.00000e-01,
        3.56087e+00,  3.68792e-03, -1.87906e+00,  8.07746e+00,
        1.03591e+00,  9.88788e-01,  1.02603e+00,  1.03121e+00,
        1.01588e+00])

In [53]:
parman.order

[[1, 'pdf', 'g a'],
 [1, 'pdf', 'g b'],
 [1, 'pdf', 'uv a'],
 [1, 'pdf', 'uv b'],
 [1, 'pdf', 'dv a'],
 [1, 'pdf', 'dv b'],
 [1, 'pdf', 'db N'],
 [1, 'pdf', 'db a'],
 [1, 'pdf', 'db b'],
 [1, 'pdf', 'ub N'],
 [1, 'pdf', 'ub a'],
 [1, 'pdf', 'ub b'],
 [1, 'pdf', 's a'],
 [1, 'pdf', 's b'],
 [1, 'pdf', 'sb N'],
 [1, 'pdf', 'sb a'],
 [1, 'pdf', 'sb b'],
 [1, 'pdf', 'sea1 N'],
 [1, 'pdf', 'sea1 a'],
 [1, 'pdf', 'sea1 b'],
 [2, 'idis', 10010],
 [2, 'idis', 10016],
 [2, 'idis', 10020],
 [2, 'idis', 10011],
 [2, 'idis', 10017]]

with the presence of the datasets normalizations parameters inside `conf`, `PARMAN` allocates new entries for the experimental nuisance parameters  $N_e$ as free parameters.

There are two important methods in `PARAMAN` that are important to mention 
- `set_new_params(parnew)`: this function takes an array of parameters of the same dimension as `parman.par` and update the pdf class accordingly 
- `gen_flat(setup=True)`: this method picks a random parameter vector using uniform sampling within the hyperbox specified by the `min` and `max` in the parameters section of `conf`

Here is an example

In [54]:
print(parman.par)
parman.gen_flat()
print(parman.par)

[-5.20900e-01  4.29360e+00 -2.14402e-01  3.04406e+00 -3.45404e-01
  4.48193e+00  3.65346e-02 -9.35028e-01  4.48545e+00  1.70043e-02
 -1.00000e+00  1.00000e+01  1.00000e+00  4.43290e+00  2.96987e-02
 -6.00000e-01  3.56087e+00  3.68792e-03 -1.87906e+00  8.07746e+00
  1.03591e+00  9.88788e-01  1.02603e+00  1.03121e+00  1.01588e+00]
[-3.72735379e-01  8.40775784e+00  5.81822344e-01  2.08383513e-01
  1.88254221e-01  7.58294892e+00  9.32298780e-03 -9.70383668e-01
  1.07676278e+00  7.57437758e-01 -7.70281703e-01  9.52078326e+00
  9.10900953e-02  8.42245565e+00  5.45637156e-03 -2.99324046e-01
  1.73945645e+00  2.35514012e-01 -1.20744855e+00  6.63574467e+00
  8.20390643e-01  9.44127252e-01  8.80063932e-01  1.15750224e+00
  8.56995446e-01]


We can retrieve the boundaries via `parman.pmin` and `parman.pmax`

In [55]:
parman.pmin

array([-1.9,  0. , -0.6,  0. , -0.6,  0. ,  0. , -1. ,  0. ,  0. , -1. ,
        0. , -0.6,  0. ,  0. , -0.6,  0. ,  0. , -1.9,  0. ,  0.8,  0.8,
        0.8,  0.8,  0.8])

In [56]:
parman.pmax

array([ 1. , 10. ,  1. , 10. ,  1. , 10. ,  1. ,  1. , 10. ,  1. ,  1. ,
       10. ,  1. , 10. ,  1. ,  1. , 10. ,  1. , -1. , 10. ,  1.2,  1.2,
        1.2,  1.2,  1.2])

### Exercise 10 (time: 5 mins)

- Use the method `gen_flat` to scan 100 random parameter vectors within the hyperbox and plot the resulting pdf conbinations $xu_v,~xd_v,~xg$ at the input scale $mu^2=1.27**2$. 
- **Hint**: use the x-range  `X = 10**np.linspace(-2,np.log10(0.9),100)`

## Residuals Manager (RESMAN)

We next build the class `RESMAN`  that glues all the parts `PARMAN` and DIS `RESIDUALS`. While this class seems to be an overkill for the current tutorial, more complex global analysis involving different reactions will requiere to have a manager for all the residuals across all the reactions 

In [57]:
class RESMAN:

    def __init__(self):

        conf['aux']    = AUX()
        conf['mellin'] = MELLIN(npts=4)
        conf['alphaS'] = ALPHAS()
        conf['eweak']  = EWEAK()
        conf['pdf']    = PDF()
        
        self.parman=PARMAN()
        self.setup_idis()
    

    def setup_idis(self):

        self.idis_tabs=READER().load_data_sets('idis')
        conf['idis tabs'] = self.idis_tabs
        self.idis_thy=THEORY()
        conf['idis'] = self.idis_thy
        self.idis_res=RESIDUALS()

    def get_residuals(self,par, initial = False):

        self.parman.set_new_params(par, initial = initial)

        data = conf['datasets']

        #--compute residuals
        res,rres,nres=[],[],[]
        if 'idis' in conf['datasets']:
            out=self.idis_res.get_residuals()
            res=np.append(res,out[0])
            rres=np.append(rres,out[1])
            nres=np.append(nres,out[2])
        return res,rres,nres

    #--aux funcs

    def get_data_info(self):

        #--compute residuals
        reaction=[]
        if 'idis' in conf['datasets']:
            out=self.idis_res.get_residuals(calc=False)
            reaction.extend(['idis' for _ in out[0]])
 
    def gen_report(self,verb=0,level=0):
        data={}
        if 'idis'    in conf['datasets']: data['idis']=self.idis_res.gen_report()
        return data

    def get_chi2(self):
        data={}
        if 'idis'    in conf['datasets']: data.update(self.idis_res.get_chi2())
        return data


### Example

In [58]:
conf={}
setup_params()
get_datasets(Q2cut=1.27**2, W2cut=10, ihera=False)
conf['order']     = 1
conf['mu20']      = 1.27**2
resman=RESMAN()

loading idis data sets 10010
loading idis data sets 10016
loading idis data sets 10020
loading idis data sets 10011
loading idis data sets 10017
loading idis data sets 10021
10010 has norm uncertainty
10016 has norm uncertainty
10020 has norm uncertainty
10011 has norm uncertainty
10017 has norm uncertainty


Notice that we have move the intialization of the various classe inside `RESMAN`. In particular the class object of `PARAMAN` is available within `RESMAN`. 

The function `get_residuals(par)` is the most relevant function of the class. It takes a vector of parmaters `par`, uses `PARAMAN` to update the pdf parameters and returns the residuals `res`, `rres`, `nres`. 

Lets, see how it works:

In [59]:
Current_par = resman.parman.par
Current_par

array([-5.20900e-01,  4.29360e+00, -2.14402e-01,  3.04406e+00,
       -3.45404e-01,  4.48193e+00,  3.65346e-02, -9.35028e-01,
        4.48545e+00,  1.70043e-02, -1.00000e+00,  1.00000e+01,
        1.00000e+00,  4.43290e+00,  2.96987e-02, -6.00000e-01,
        3.56087e+00,  3.68792e-03, -1.87906e+00,  8.07746e+00,
        1.03591e+00,  9.88788e-01,  1.02603e+00,  1.03121e+00,
        1.01588e+00])

In [60]:
res,rres,nres = resman.get_residuals(Current_par)

In [61]:
print(len(res))
print(len(rres))
print(len(nres))

1503
26
6


The method `get_chi2` can be used to query the current $\chi^2$ values computed as 
`chi2=np.sum(res**2)`

In [62]:
resman.get_chi2()['idis']

{10010: {'npts': 222, 'chi2': 188.24660499575094},
 10016: {'npts': 348, 'chi2': 408.041006884065},
 10020: {'npts': 274, 'chi2': 360.68291375671726},
 10011: {'npts': 231, 'chi2': 145.5293389580629},
 10017: {'npts': 254, 'chi2': 281.43458642100154},
 10021: {'npts': 174, 'chi2': 168.07703204282362}}

After the method `resman.get_residuals(par)` is called, the `thy` columns of the tables are updated.   

In [63]:
pd.DataFrame(resman.idis_res.tabs[10010])

,col,X,Q2,obs,target,lepton beam,current,units,value,stat_u,...,W2,idx,thy,N,residuals,r-residuals,Shift,alpha,shift,prediction
0,SLAC,0.075,1.66,F2,p,e,NC,1.0,0.35448,0.01028,...,21.354902,6,0.350662,1.0,1.417800,0.0,0.0,0.011266,0.0,0.338507
1,SLAC,0.077,2.01,F2,p,e,NC,1.0,0.37937,0.01328,...,24.975465,10,0.357514,1.0,2.127474,0.0,0.0,0.016099,0.0,0.345120
2,SLAC,0.080,1.67,F2,p,e,NC,1.0,0.35137,0.01476,...,20.086569,12,0.352053,1.0,0.649433,0.0,0.0,0.017739,0.0,0.339850
3,SLAC,0.083,1.76,F2,p,e,NC,1.0,0.34599,0.00588,...,20.326388,17,0.354406,1.0,0.350645,0.0,0.0,0.011037,0.0,0.342120
4,SLAC,0.083,2.39,F2,p,e,NC,1.0,0.35900,0.01185,...,27.286750,18,0.363825,1.0,0.516304,0.0,0.0,0.015082,0.0,0.351213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,SLAC,0.633,17.69,F2,p,e,NC,NaN,0.05403,0.00259,...,11.137856,510,0.054184,1.0,0.387468,0.0,0.0,0.004451,0.0,0.052305
218,SLAC,0.658,22.12,F2,p,e,NC,NaN,0.04110,0.00148,...,12.378590,528,0.042058,1.0,0.189698,0.0,0.0,0.002635,0.0,0.040600
219,SLAC,0.667,22.93,F2,p,e,NC,NaN,0.03799,0.00125,...,12.329380,533,0.038588,1.0,0.263746,0.0,0.0,0.002804,0.0,0.037250
220,SLAC,0.708,23.66,F2,p,e,NC,NaN,0.02551,0.00184,...,10.639648,570,0.025972,1.0,0.190411,0.0,0.0,0.002300,0.0,0.025072


### Exercise 11 (time: 5 mins)

- Use `RESMAN` to compute 10 times the residuals for different parameters generated by `parman.gen_flat()`
- For each run print the first 3 entries of the current parameters as well a the total $\chi^2$ 



## First steps for Machine learning:  Maximun likelihood

In [64]:
class MAXLIKE:
  
    def __init__(self):

        self.resman=RESMAN()
        self.parman=self.resman.parman
        self.set_display()
        
  
    def get_residuals(self,par):
        res,rres,nres=self.resman.get_residuals(par)
        self.cnt+=1
        self.print_status(res,rres,nres)
        if len(rres)!=0: res=np.append(res,rres)
        if len(nres)!=0: res=np.append(res,nres)
        return res

    def checklimits(self):

        for k in conf['params']:
            for kk in conf['params'][k]:
                if conf['params'][k][kk]['fixed']!=False: continue
                p=conf['params'][k][kk]['value']
                pmin=conf['params'][k][kk]['min']
                pmax=conf['params'][k][kk]['max']
                if  p<pmin or p>pmax:
                    print ('%s-%s out of limits. '%(k,kk))
                    sys.exit()

        for k in conf['datasets']:
            for kk in conf['datasets'][k]['norm']:
                p=conf['datasets'][k]['norm'][kk]['value']
                pmin=conf['datasets'][k]['norm'][kk]['min']
                pmax=conf['datasets'][k]['norm'][kk]['max']
                if  p<pmin or p>pmax: 
                    print ('%s-%s out of limits. '%(k,kk))
                    sys.exit()


    def get_bounds(self):
        order=self.parman.order

        bounds_min=[]
        bounds_max=[]
        for entry in order:
            i,k,kk=entry
            if  i==1:
                p=conf['params'][k][kk]['value']
                pmin=conf['params'][k][kk]['min']
                pmax=conf['params'][k][kk]['max']
                if p<pmin or p>pmax: 
                    msg='%s/%s outsize the limits %f %f %f'%(k,kk,p,pmin,pmax)
                    raise ValueError(msg)
                bounds_min.append(conf['params'][k][kk]['min'])
                bounds_max.append(conf['params'][k][kk]['max'])
            elif i==2:
                p=conf['datasets'][k]['norm'][kk]['value']
                pmin=conf['datasets'][k]['norm'][kk]['min']
                pmax=conf['datasets'][k]['norm'][kk]['max']
                if p<pmin or p>pmax: 
                    msg='%s/%s outsize the limits %f %f %f'%(k,kk,p,pmin,pmax)
                    raise ValueError(msg)
                bounds_min.append(conf['datasets'][k]['norm'][kk]['min'])
                bounds_max.append(conf['datasets'][k]['norm'][kk]['max'])

        return (bounds_min, bounds_max)

    def get_guess(self):
        if  conf['flat par']: 
            guess=self.parman.gen_flat()
        else:
            guess=self.parman.par
        return guess


    def run(self):
        #--current value must be within min and max
        self.checklimits()

        #--setups  
        if 'guess' not in conf or conf['guess'][0]==None:
            guess=self.get_guess()           
        else:
            guess=conf['guess']
 
        bounds=self.get_bounds()
        self.set_counters()
        self.parman.set_new_params(guess,initial=True)
        
        fit = least_squares(self.get_residuals, guess,bounds=bounds,method='trf',ftol=conf['ftol'])
        self.sol=fit.x
        #self.sol=leastsq(self.get_residuals, guess,ftol=conf['ftol'])[0]
    
    
    def set_counters(self):
        self.chi2tot=1e1000
        self.dchi2=0
        self.t0 = time.time()
        self.cnt=0
        
    def set_display(self):

        children=[]
        children.append(widgets.HTMLMath(value='',placeholder='',description='',)) #--summary
        children.append(widgets.HTMLMath(value='',placeholder='',description='',)) #--sum rules
        children.append(widgets.HTMLMath(value='',placeholder='',description='',)) #--chi2 per exp
        children.append(widgets.HTMLMath(value='',placeholder='',description='',)) #--params           
        #children.append(widgets.Output())
    
        self.tab = widgets.Tab()
        self.tab.children = children
  
    def print_status(self,res,rres,nres):

        #--update status parameters  
        shifts=self.parman.shifts
        etime = (time.time()-self.t0)/60
        npts=res.size
        chi2=np.sum(res**2)
        rchi2=np.sum(rres**2)
        nchi2=np.sum(nres**2)
        chi2tot=chi2+rchi2+nchi2
        dchi2=chi2tot-self.chi2tot
        if  shifts>2: 
            if  chi2tot<self.chi2tot:
                self.dchi2=self.chi2tot-chi2tot
                self.chi2tot=chi2tot
  

        status={}
        status['var']=['count'
                      ,'elapsed time(mins)'
                      ,'shifts'
                      ,'npts'
                      ,'chi2'
                      ,'rchi2'
                      ,'nchi2' 
                      ,'chi2tot'
                      ,'dchi2(iter)' 
                      ,'dchi2(local)' 
                      ]
        status['value']=['%d'%self.cnt
                      ,'%0.2f'%etime
                      ,'%d'%shifts
                      ,'%d'%npts
                      ,'%0.2f'%chi2
                      ,'%0.2f'%rchi2
                      ,'%0.2f'%nchi2
                      ,'%0.2f'%chi2tot
                      ,'%0.2f'%self.dchi2
                      ,'%0.2f'%dchi2
                      ]
        status=pd.DataFrame(status,index=None)
        self.tab.set_title(0, 'summary')
        msg='JAM FITTER <br />'
        msg+=status.to_html(col_space=200,index=False)
        self.tab.children[0].value = msg

        
        #--sumrules
        status={}
        status['var']=[_ for _ in conf['pdf'].sr]
        status['value']=['%0.2f'%conf['pdf'].sr[_] for _ in conf['pdf'].sr]
        status=pd.DataFrame(status,index=None)
        self.tab.set_title(1, 'sum rules')
        self.tab.children[1].value = status.to_html(col_space=200,index=False)

        #--report from resman 
        msg=self.resman.gen_report()['idis']
        self.tab.set_title(2, 'chi2 per exp')
        self.tab.children[2].value = msg

        #--report from parman
        msg = self.parman.gen_report()
        self.tab.set_title(3, 'parameters')
        self.tab.children[3].value = msg
        


### Example

In [65]:
conf={}
setup_params()
get_datasets(Q2cut=1.27**2, W2cut=10, ihera=False)
conf['order']     = 1
conf['mu20']      = 1.27**2
conf['flat par']  = False
conf['ftol']      = 1e-2
conf['bootstrap'] = False
conf['guess']     = [None]

maxlike=MAXLIKE()
thread = threading.Thread(target=maxlike.run)
display(maxlike.tab)
thread.start()

loading idis data sets 10010
loading idis data sets 10016
loading idis data sets 10020
loading idis data sets 10011
loading idis data sets 10017
loading idis data sets 10021
10010 has norm uncertainty
10016 has norm uncertainty
10020 has norm uncertainty
10011 has norm uncertainty
10017 has norm uncertainty


### Comments

The class `MAXLIKE` uses a gradient descent algoritm  `scipy.optimize.least_squares` to carryout the maximun likelihood.  There are few important parameters in `conf` that controls the fitting

- `conf['ftol']`: this controls the stopping of the minimization by limiting the relative change in $\chi^2$
- `conf['flat par']`: If `True`, the staring guessing parameters are picked randomly within the hyperbox. Otherwise, the minimization starts with the values specified in `conf['params']`.
- `conf['bootstrap']`: If `True`, the experimental data is shifted with gaussian noise within the quoted uncorrelated uncertaies.
- `conf['guess']`: If !=[None], MAXLIKE will take its value as a starting point for the minimization

Once the minimization ends, we can access optimized vector of parameters via `maxlike.sol`. 

In [ ]:
par1=maxlike.sol  #--lets save this vector of parameters 
par1

In [ ]:
conf={}
setup_params()
get_datasets(Q2cut=1.27**2, W2cut=10, ihera=False)
conf['order']     = 1
conf['mu20']      = 1.27**2
resman=RESMAN()

In [ ]:
resman.get_residuals(par1)

In [ ]:
tabs=resman.idis_tabs
nrows,ncols = 1,1
fig = py.figure(figsize=(ncols*7,nrows*5))
ax = py.subplot(nrows,ncols,1)
H,C={},{}
for idx in tabs:
    tab=tabs[idx]
    if 'HERA' in tab['col'][0]: col='HERA'
    else: col=tab['col'][0]
    if col not in C:
        p,=ax.plot(tab['X'],tab['Q2'],ls='',marker='.',markersize=10)
        C[col]=p.get_color()
        H[col]=p
    else:
        ax.plot(tab['X'],tab['Q2'],ls='',marker='.',color=C[col],markersize=10)
    
ax.tick_params(axis='both', which='major', labelsize=20)
ax.legend([H[_] for _ in H],[r'$\rm %s$'%_ for _ in H],loc=2,fontsize=20,framealpha=1)
ax.set_xlabel(r'$x_{\rm bj}$',size=25)
ax.set_ylabel(r'$Q^2~({\rm GeV}^2)$',size=25)
ax.semilogx()
ax.semilogy();

In [ ]:
tabs=resman.idis_tabs
nrows,ncols = len(tabs.keys())/3,3
fig = py.figure(figsize=(ncols*7,nrows*5))

cnt=0
for idx in tabs:
    cnt+=1
    ax = py.subplot(nrows,ncols,cnt)
    tab=tabs[idx]
    ax.errorbar(tab['X'],tab['value']/tab['thy'],tab['alpha']/tab['thy'],fmt='.')    
    ax.set_ylim(0.5,1.5)    
    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.semilogx()
    if cnt==1: ax.set_ylabel(r'$\rm data/theory$',size=30)
    msg=r'$\rm %s~%s$'%(tab['col'][0],tab['target'][0])
    ax.text(0.1,0.8,msg,transform=ax.transAxes,size=30)
ax.set_xlabel(r'$x_{\rm bj}$',size=30);

### Exercise 12.A (time: 5 mins)
- Plot $xu_v$, $xd_v$, $d/u$, $x(\bar{d}+\bar{u})$,$x(\bar{d}-\bar{u})$, $R_s=(\bar{d}+\bar{u})/(s+\bar{s})$, $xg$
- **hint**: use the method  `conf['pdf'].get_xf(x,mu2,flav)`

### Exercise 12.B (time: 10 mins)

- Include the HERA data sets and run the fitter 
- Plot data/theory for all experiments
- Plot the pdfs (as above) from the previous run and the new run
- **hint**: 
  + use `get_datasets(Q2cut=1.27**2, W2cut=10, ihera=True)`
  + assuming you have the two sets of parameters stored as par1 and par2, when making the pdfs plots, use the method `    resman.parman.set_new_params(par)`  to update the `PDF` class